In [1]:
import pandas as pd
import pyodbc
import roster_match_6 as roster_match
from useful_functions import fix_me
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nameparser import HumanName

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
username = 'vigrose'
password = 'Ravenclaw~10946'
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

In [8]:
QUERY = \
'''
SELECT 
party.PARTY_ID as me, 
addr.EMAIL_STATUS as status 
FROM AMAEDW.EMAIL_ADDR addr INNER JOIN AMAEDW.PARTY_EMAIL party 
ON party.EMAIL_ID=addr.EMAIL_ID AND party.THRU_DT IS NULL AND party.PURPOSE_TYPE_ID = 315
'''

In [12]:
query = \
    """
    SELECT DISTINCT 
    PE.PARTY_ID,
    E.EMAIL_ADDR
    FROM 
    AMAEDW.EMAIL_ADDR E, AMAEDW.PARTY_EMAIL PE
    WHERE  
    E.EMAIL_ID = PE.EMAIL_ID
    AND
    PE.THRU_DT IS NULL
    AND
    PE.PURPOSE_TYPE_ID = 315
    AND 
    E.EMAIL_STATUS = 'valid'
    """

In [9]:
emails = pd.read_sql(con=AMAEDW, sql=QUERY)
emails.head()

,ME,STATUS
0,6523014,invalid
1,12166055,invalid
2,10439036,invalid
3,6108501,invalid
4,4555060,invalid


In [17]:
emails.drop_duplicates('ME').groupby('STATUS').count()

,ME
STATUS,
invalid,136361
valid,677872


In [13]:
EMAILS = pd.read_sql(con=AMAEDW, sql=query)
EMAILS.head()

,PARTY_ID,EMAIL_ADDR
0,1798001,kiddoc2b@hotmail.com
1,1798002,hortonaj@msn.com
2,1798003,acooley@apollomd.com
3,1798004,gfolsom@orthokc.com
4,1798007,dey90@hotmail.com


In [15]:
EMAILS

,PARTY_ID,EMAIL_ADDR
0,1798001,kiddoc2b@hotmail.com
1,1798002,hortonaj@msn.com
2,1798003,acooley@apollomd.com
3,1798004,gfolsom@orthokc.com
4,1798007,dey90@hotmail.com
...,...,...
677874,16428397,todd.sorensen33@gmail.com
677875,16428610,patelarpan03@gmail.com
677876,16429153,cons10eu@gmail.com
677877,16430715,valerie@thehearinghealthcenter.com


In [7]:
EMAILS['USERNAME'] = [x.split('@')[0] for x in EMAILS.EMAIL_ADDR]
EMAILS['DOMAIN'] = [x.split('@')[1] for x in EMAILS.EMAIL_ADDR]

In [13]:
domains = EMAILS.groupby('DOMAIN').count().sort_values('ME', ascending=False)

In [97]:
EMAILS.to_csv('../../Data/Email/emails_20201130.csv', index=False)

In [36]:
MAYO_EMAILS = EMAILS[EMAILS.DOMAIN=='mayo.edu']

In [79]:
UT_EMAILS

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME,DOMAIN
6744,00102140788,david.breland@uth.tmc.edu,No Usage,david.breland,uth.tmc.edu
6745,00102140788,lea.l.roberts@uth.tmc.edu,No Usage,lea.l.roberts,uth.tmc.edu
8871,00102171217,kerollos.a.shaker@uth.tmc.edu,Single Sign-On,kerollos.a.shaker,uth.tmc.edu
9361,00102180585,omar.m.ismail@uth.tmc.edu,No Usage,omar.m.ismail,uth.tmc.edu
9362,00102180585,omar.m.ismail@uth.tmc.edu,SSO Email ...,omar.m.ismail,uth.tmc.edu
...,...,...,...,...,...
3765671,95703790039,igor.d.gregoric@uth.tmc.edu,Single Sign-On,igor.d.gregoric,uth.tmc.edu
3767158,96501880041,jayeshkumar.a.patel@uth.tmc.edu,AMA Web Site,jayeshkumar.a.patel,uth.tmc.edu
3767159,96501880041,jayeshkumar.a.patel@uth.tmc.edu,No Usage,jayeshkumar.a.patel,uth.tmc.edu
3767160,96501880041,jayeshkumar.a.patel@uth.tmc.edu,Preferred E-mail,jayeshkumar.a.patel,uth.tmc.edu


In [78]:
UT_EMAILS = EMAILS[EMAILS.DOMAIN=='uth.tmc.edu']

In [92]:
UCLA_EMAILS = EMAILS[EMAILS.DOMAIN=='mednet.ucla.edu']

In [80]:
ut = pd.read_csv("../../Data/Rosters/UTHealth_Scrape_2020-10-15.csv")

In [23]:
mayo = pd.read_csv("../../Data/Rosters/Mayo_clinic_Scrape_2020-09-15.csv")

In [93]:
ucla = pd.read_csv("../../Data/Rosters/UCLA_Health_2020-11-01.csv")

In [95]:
ucla['NAME']=ucla.name
ucla = split_names(ucla)

In [82]:
ut = split_names(ut)

In [27]:
mayo = split_names(mayo)

In [87]:
ut_emails = []
for row in ut.itertuples():
    ut_emails.append(row.LINK.split('/')[-2].replace('-','.'))

In [96]:
UCLA_EMAILS

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME,DOMAIN
5387,00102111915,snwaobi@mednet.ucla.edu,AMA Web Site,snwaobi,mednet.ucla.edu
5388,00102111915,snwaobi@mednet.ucla.edu,Morning Rounds Daily Edition ...,snwaobi,mednet.ucla.edu
5389,00102111915,snwaobi@mednet.ucla.edu,No Usage,snwaobi,mednet.ucla.edu
5390,00102111915,snwaobi@mednet.ucla.edu,Preferred E-mail,snwaobi,mednet.ucla.edu
6115,00102130626,jwfrederick@mednet.ucla.edu,AMA Web Site,jwfrederick,mednet.ucla.edu
...,...,...,...,...,...
3763804,95701860064,jmarijic@mednet.ucla.edu,Preferred E-mail,jmarijic,mednet.ucla.edu
3764415,95702030065,jmaletkovic@mednet.ucla.edu,AMA Web Site,jmaletkovic,mednet.ucla.edu
3764416,95702030065,jmaletkovic@mednet.ucla.edu,Morning Rounds Daily Edition ...,jmaletkovic,mednet.ucla.edu
3764417,95702030065,jmaletkovic@mednet.ucla.edu,No Usage,jmaletkovic,mednet.ucla.edu


In [89]:
UT_EMAILS[UT_EMAILS.USERNAME.isin(ut_emails)].sort_values('PURPOSE_USG_DESC', ascending=False).drop_duplicates('ME')

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME,DOMAIN
3767161,96501880041,jayeshkumar.a.patel@uth.tmc.edu,Single Sign-On,jayeshkumar.a.patel,uth.tmc.edu
2564653,04802931366,william.b.perkison@uth.tmc.edu,Single Sign-On,william.b.perkison,uth.tmc.edu
3449996,60501730222,baha.m.sibai@uth.tmc.edu,Single Sign-On,baha.m.sibai,uth.tmc.edu
2558101,04802781691,karl.schmitt@uth.tmc.edu,Single Sign-On,karl.schmitt,uth.tmc.edu
2560594,04802840417,kimberly.c.smith@uth.tmc.edu,Single Sign-On,kimberly.c.smith,uth.tmc.edu
...,...,...,...,...,...
2570989,04804061199,ryan.s.kitagawa@uth.tmc.edu,No Usage,ryan.s.kitagawa,uth.tmc.edu
2597060,04812050813,alisha.y.young@uth.tmc.edu,No Usage,alisha.y.young,uth.tmc.edu
2592887,04804970512,siddharth.k.prakash@uth.tmc.edu,No Usage,siddharth.k.prakash,uth.tmc.edu
2574334,04804121752,li.xiong@uth.tmc.edu,No Usage,li.xiong,uth.tmc.edu


In [53]:
mayo_emails = []
for row in mayo.itertuples():
    first = row.FIRST_NAME.lower()
    last = row.LAST_NAME.lower().replace(' ','')
    email = f'{last}.{first}'
    mayo_emails.append(email)
    mayo_emails.append(f'{first[0]}{last}')
    if 'biographies/' in row.LINK:
        link = row.LINK.split('biographies/')[1]
        split_link = link.split('-')
        mayo_emails.append(f'{split_link[0]}.{split_link[1]}')
        mayo_emails.append(f'{split_link[1][0]}{split_link[0]}')

In [54]:
MAYO_EMAILS[MAYO_EMAILS.USERNAME.isin(mayo_emails)].sort_values('PURPOSE_USG_DESC', ascending=False).drop_duplicates('ME')

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME,DOMAIN
1301064,02604971457,bothun.erick@mayo.edu,Single Sign-On,bothun.erick,mayo.edu
1098850,02405860506,hieken.tina@mayo.edu,Single Sign-On,hieken.tina,mayo.edu
1097912,02405831361,lust.john@mayo.edu,Single Sign-On,lust.john,mayo.edu
3217843,49508820070,nehra.vandana@mayo.edu,Single Sign-On,nehra.vandana,mayo.edu
1368968,02803951422,mauck.william@mayo.edu,Single Sign-On,mauck.william,mayo.edu
...,...,...,...,...,...
3148692,41801060011,kourelis.taxiarchis@mayo.edu,No Usage,kourelis.taxiarchis,mayo.edu
1304767,02608060017,fazzio.robert@mayo.edu,No Usage,fazzio.robert,mayo.edu
1305853,02608130996,ahmed.safia@mayo.edu,Morning Rounds Daily Edition ...,ahmed.safia,mayo.edu
1575464,03401920110,driscoll.colin@mayo.edu,Morning Rounds Daily Edition ...,driscoll.colin,mayo.edu


In [56]:
MAYO_EMAILS[MAYO_EMAILS.USERNAME.isin(mayo_emails)==False].drop_duplicates('EMAIL_ADDR')

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME,DOMAIN
828,00102011741,hess.erik@mayo.edu,AMA Web Site,hess.erik,mayo.edu
4273,00102091396,gladden.james@mayo.edu,AMA Web Site,gladden.james,mayo.edu
4562,00102100816,arun.janani@mayo.edu,AMA Web Site,arun.janani,mayo.edu
6816,00102141059,partain.daniel@mayo.edu,AMA Web Site,partain.daniel,mayo.edu
7024,00102141733,partain.paige@mayo.edu,AMA Web Site,partain.paige,mayo.edu
...,...,...,...,...,...
3765415,95702930082,natasa.milosavljevic@mayo.edu,Membership Email. ...,natasa.milosavljevic,mayo.edu
3765482,95702950075,jevremovic.dragan@mayo.edu,AMA Web Site,jevremovic.dragan,mayo.edu
3765757,95704100010,stojanovska.irina@mayo.edu,No Usage,stojanovska.irina,mayo.edu
3765947,95705770027,sprung.juraj@mayo.edu,AMA Web Site,sprung.juraj,mayo.edu


In [51]:
len(MAYO_EMAILS.drop_duplicates('EMAIL_ADDR'))

4163

In [55]:
mayo_emails

['aakre.christopher',
 'caakre',
 'aakre.christopher',
 'caakre',
 'abate.ejigayehu',
 'eabate',
 'abate.ejigayehu',
 'eabate',
 'abboud.charles',
 'cabboud',
 'abboud.charles',
 'cabboud',
 'abcejo.arnoley',
 'aabcejo',
 'abcejo.arnoley',
 'aabcejo',
 'abdel.matthew',
 'mabdel',
 'abdel.matthew',
 'mabdel',
 'abel.martin',
 'mabel',
 'abel.martin',
 'mabel',
 'abenstein.john',
 'jabenstein',
 'abenstein.john',
 'jabenstein',
 'abode-iyamah.kingsley',
 'kabode-iyamah',
 'abode.iyamah',
 'iabode',
 'ezzeddine.omar',
 'oezzeddine',
 'abou.ezzeddine',
 'eabou',
 'abraham.neena',
 'nabraham',
 'abraham.neena',
 'nabraham',
 'abril.andy',
 'aabril',
 'abril.andy',
 'aabril',
 'absah.imad',
 'iabsah',
 'absah.imad',
 'iabsah',
 'abudabrh.abd',
 'aabudabrh',
 'abu.dabrh',
 'dabu',
 'abudayyeh.barham',
 'babudayyeh',
 'abu.dayyeh',
 'dabu',
 'abulebdeh.haitham',
 'habulebdeh',
 'abu.lebdeh',
 'labu',
 'accurso.joseph',
 'jaccurso',
 'accurso.joseph',
 'jaccurso',
 'achem.sami',
 'sachem',
 'ac

In [58]:
ppd = pd.read_csv('../../Data/PPD/ppd_data_20201121.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [69]:
TESTING = pd.merge(PPD, mayo, on=['FIRST_NAME','LAST_NAME'], suffixes=['_ppd', '_mayo']).drop_duplicates('LINK', keep=False)

In [66]:
PPD = ppd[['ME', 'MAILING_NAME','LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'POLO_CITY', 'POLO_STATE' ]]
PPD['ME']=fix_me(PPD.ME)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [73]:
email_add = pd.merge(TESTING, EMAILS, on='ME', how='left').fillna('None')

In [74]:
email_add[email_add['EMAIL_ADDR']=='None']

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME_ppd,SUFFIX_ppd,POLO_CITY,POLO_STATE,NAME,MIDDLE_NAME_mayo,...,NICKNAME,TITLE,LINK,SPECIALTY,CITY,DATE,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME,DOMAIN
0,00102020693,KRISTOPHER W CUMMINGS MD,CUMMINGS,KRISTOPHER,W,None,PHOENIX,AZ,Kristopher W. Cummings,W.,...,,,/biographies/cummings-kristopher-w-m-d/bio-201...,Radiologist,"Phoenix, AZ",2020-09-15T00:01:22.000000Z,None,None,None,None
1,00102021231,JILL ADAMSKI MD,ADAMSKI,JILL,None,None,SCOTTSDALE,AZ,Jill Adamski,,...,,,/biographies/adamski-jill-m-d-ph-d/bio-20087333,Pathologist,"Phoenix, AZ",2020-09-15T00:00:33.000000Z,None,None,None,None
84,00375010542,HELENE R LABONTE DO,LABONTE,HELENE,R,None,SCOTTSDALE,AZ,Helene R. Labonte,R.,...,,,/biographies/labonte-helene-r-d-o/bio-20054833,Geriatrician,"Phoenix, AZ",2020-09-15T00:02:40.000000Z,None,None,None,None
99,00502121235,ZLATKO DEVCIC MD,DEVCIC,ZLATKO,None,None,JACKSONVILLE,FL,Zlatko Devcic,,...,,,/biographies/devcic-zlatko-m-d/bio-20435423,Hospitalist,"Jacksonville, FL",2020-09-15T00:01:30.000000Z,None,None,None,None
128,00512050093,EJIGAYEHU G ABATE MD,ABATE,EJIGAYEHU,GETAHUNE,None,JACKSONVILLE,FL,Ejigayehu G. Abate,G.,...,,,/biographies/abate-ejigayehu-g-m-d/bio-20055425,Endocrinologist,"Jacksonville, FL",2020-09-15T00:00:32.000000Z,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4444,75901960038,ALEKSANDRA K KUKLA MD,KUKLA,ALEKSANDRA,KINGA,None,ROCHESTER,MN,Aleksandra Kukla,,...,,,/biographies/kukla-aleksandra-m-d/bio-20241280,Nephrologist,"Rochester, MN",2020-09-15T00:02:40.000000Z,None,None,None,None
4531,87501020140,MHD L MANINI MD,MANINI,MHD,LOUAI,None,ROCHESTER,MN,Mhd Louai Manini,LOUAI,...,,,/biographies/manini-mhd-louai-m-d/bio-20204902,Pediatrician,"Rochester, MN",2020-09-15T00:02:52.000000Z,None,None,None,None
4557,87501940206,GHADA M MITRI MD,MITRI,GHADA,MITRI,None,JACKSONVILLE,FL,Ghada Mitri,,...,,,/biographies/mitri-ghada-m-d/bio-20055659,None,"Jacksonville, FL",2020-09-15T00:03:04.000000Z,None,None,None,None
4585,89102000065,TEERIN LIEWLUCK MD,LIEWLUCK,TEERIN,None,None,ROCHESTER,MN,Teerin Liewluck,,...,,,/biographies/liewluck-teerin-m-d/bio-20152864,Neurologist,"Rochester, MN",2020-09-15T00:02:45.000000Z,None,None,None,None


In [75]:
cinc = pd.read_csv('../../Data/Rosters/Cincinnati_Stuff.csv')
dinc_do = pd.read_csv('../../Data/Rosters/Cincinnati_DOs.csv')
# ppd = pd.read_csv('../../Data/PPD/ppd_data_20200718.csv')

In [19]:
PPD = ppd[['ME', 'MAILING_NAME','LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'POLO_CITY', 'POLO_STATE' ]]
PPD['ME']=fix_me(PPD.ME)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [64]:
cinc = cinc.fillna('None')
dinc_do = dinc_do.fillna('None')
# PPD = PPD.fillna('None')

In [724]:
PPD_TEST

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,GENDER,PRIM_SPEC_CD,...,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM
0,102000013,MICHAEL J STANFORD MD,STANFORD,MICHAEL,JAMES,NaN,1,1972.0,1.0,EM,...,1,2000,NaN,NaN,EM,1682,EMERGENCY MEDICINE,FM,1691,FAMILY MEDICINE
1,102000021,MERI K MARTIN MD,MARTIN,MERI,KING,NaN,1,1955.0,2.0,FOP,...,1,2001,RALEIGH,NC,FOP,1692,FORENSIC PATHOLOGY,US,1883,UNSPECIFIED
2,102000030,MARK T JACKSON MD,JACKSON,MARK,THOMAS,NaN,1,1973.0,1.0,PD,...,1,2000,ASHEVILLE,NC,PD,1800,PEDIATRICS,US,1883,UNSPECIFIED
3,102000048,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,1,1960.0,2.0,IM,...,1,2000,SUN CITY WEST,AZ,IM,1724,INTERNAL MEDICINE,US,1883,UNSPECIFIED
4,102000056,RAJIV B GALA MD,GALA,RAJIV,BABULAL,NaN,1,1973.0,1.0,OBG,...,1,2000,NEW ORLEANS,LA,OBG,1769,OBSTETRICS & GYNECOLOGY,US,1883,UNSPECIFIED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283761,96501960028,ROBERT SAKULANDA MD,SAKULANDA,ROBERT,NaN,NaN,1,1969.0,1.0,FM,...,965,1996,NaN,NaN,FM,1691,FAMILY MEDICINE,US,1883,UNSPECIFIED
1283762,96501960036,KAMISHELE MAILA MD,MAILA,KAMISHELE,NaN,NaN,1,1967.0,1.0,IM,...,965,1996,LIMA,OH,IM,1724,INTERNAL MEDICINE,US,1883,UNSPECIFIED
1283763,96501960044,MWIZA GAUSI MD,GAUSI,MWIZA,NaN,NaN,1,1971.0,1.0,IM,...,965,1998,DANVILLE,VA,IM,1724,INTERNAL MEDICINE,US,1883,UNSPECIFIED
1283764,96501960052,MASIKU A MDALA-GAUSI MD,MDALA-GAUSI,MASIKU,AGATHA,NaN,1,1971.0,2.0,US,...,965,1996,WINSTON SALEM,NC,US,1883,UNSPECIFIED,US,1883,UNSPECIFIED


In [731]:
matched_2, missing_2, duplicates_2 = roster_match.match(dinc_do, ppd_spec)

143 duplicates found.
7 names missing.
2 duplicates found.
4 unique matches found.
24
28
4 new matches confirmed. 28 total matches confirmed.
0 duplicates found.
1 unique matches found.
28
29
1 new matches confirmed. 29 total matches confirmed.
0 duplicates found.
0 unique matches found.
0 new matches confirmed. 29 total matches confirmed.
0 duplicates found.
0 unique matches found.
0 new matches confirmed. 29 total matches confirmed.


In [734]:
matched_2.drop_duplicates('ME')

,CITY_MATCH,DEGREE,DEPARTMENT,DEPARTMNET,DEPARTMNETS,DESC,DESC_PRIM,DO,Email,FIRST_NAME,...,NAME,NICKNAME,POLO_CITY,POLO_STATE,SPEC_MATCH,STATE_MATCH,SUFFIX_x,SUFFIX_y,TITLE,TITLES
132,NaN,DO\r\n,UC Department of Neurosurgery/UC Department of...,Neurosurgery,UC Department of Neurosurgery/UC Department of...,GENERAL SURGERY,GENERAL SURGERY,True,sudhakar.vadivelu@cchmc.org,SUDHAKAR,...,Sudhakar Vadivelu,,NaN,NaN,NaN,NaN,NaN,,"Faculty Neurosurgeon, Division of Pediatric Ne...","[Faculty Neurosurgeon Pediatric Neurosurgery, ..."
140,NaN,DO,UC Department of Anesthesia/UC Department of A...,Anesthesia,UC Department of Anesthesia/UC Department of A...,ANESTHESIOLOGY,ANESTHESIOLOGY,True,ali.kandil@cchmc.org,ALI,...,Ali I. Kandil,,CINCINNATI,OH,NaN,NaN,NaN,,"Anesthesiologist, Department of Anesthesiology...","[Anesthesiologist Anesthesiology, Intra-operat..."
143,NaN,DO\r\n,UC Department of Pediatrics/UC Department of P...,Pediatrics,UC Department of Pediatrics/UC Department of P...,SPORTS MEDICINE (PEDIATRICS),SPORTS MEDICINE (PEDIATRICS),True,kate.berz@cchmc.org,KATE,...,Kate E. Berz,,CINCINNATI,OH,NaN,NaN,NaN,,"Assistant Program Director, Pediatric Sports M...","[Assistant Pediatric Sports Medicine , Emer..."
144,NaN,DO\r\n,UC Department of Surgery/UC Department of Surgery,Surgery,UC Department of Surgery/UC Department of Surgery,PEDIATRIC SURGERY (SURGERY),PEDIATRIC SURGERY (SURGERY),True,beth.rymeski@cchmc.org,BETH,...,Beth Rymeski,,CINCINNATI,OH,NaN,NaN,NaN,,"Pediatric Surgeon, Pediatric General and Thora...",[Pediatric SurgeonPediatric General and Thorac...
147,NaN,DO | Cincinnati Children's Hospital Medical Ce...,None,,None,PEDIATRIC CRITICAL CARE MEDICINE,PEDIATRIC CRITICAL CARE MEDICINE,True,danielle.maholtz@cchmc.org,DANIELLE,...,Danielle Maholtz,,NaN,NaN,NaN,NaN,NaN,,"Clinical Fellow, Division of Critical Care Med...",[ Critical Care Medicine]
148,NaN,DO\r\n,UC Department of Pediatrics/UC Department of P...,Pediatrics,UC Department of Pediatrics/UC Department of P...,PEDIATRIC HEMATOLOGY-ONCOLOGY,PEDIATRIC HEMATOLOGY-ONCOLOGY,True,brian.turpin@cchmc.org,BRIAN,...,Brian K. Turpin,,CINCINNATI,OH,NaN,NaN,NaN,,"Member, Division of Oncology/Burnet Campus/Lib...","[ Oncology, Burnet Campus, Liberty Campus]"
149,NaN,DO | Cincinnati Children's Hospital Medical Ce...,None,,None,PEDIATRIC HEMATOLOGY-ONCOLOGY,PEDIATRIC HEMATOLOGY-ONCOLOGY,True,laquita.jones@cchmc.org,LAQUITA,...,LaQuita Jones,,CINCINNATI,OH,NaN,NaN,NaN,,"Member, Division of Oncology",[ Oncology]
150,NaN,DO\r\n,UC Department of Anesthesia/UC Department of A...,Anesthesia,UC Department of Anesthesia/UC Department of A...,RHEUMATOLOGY,RHEUMATOLOGY,True,lisa.francis@cchmc.org,LISA,...,Lisa J. Francis,,NaN,NaN,NaN,NaN,NaN,,"Attending Anesthesiologist, Department of Anes...","[ Anesthesiologist Anesthesia, Intraoperative ..."
151,NaN,DO\r\n,UC Department of Pediatrics/UC Department of P...,Pediatrics,UC Department of Pediatrics/UC Department of P...,PEDIATRIC EMERGENCY MEDICINE (PEDIATRICS),PEDIATRIC EMERGENCY MEDICINE (PEDIATRICS),True,lauren.riney@cchmc.org,LAUREN,...,Lauren C. Riney,,CINCINNATI,OH,NaN,NaN,NaN,,"Attending Physician, Division of Emergency Med...","[ Emergency Medicine, Tracheitis]"
152,NaN,DO | Cincinnati Children's Hospital Medical Ce...,None,,None,PEDIATRIC EMERGENCY MEDICINE (PEDIATRICS),PEDIATRIC EMERGENCY MEDICINE (PEDIATRICS),True,bryce.willen@cchmc.org,BRYCE,...,Bryce L. Willen,,CINCINNATI,OH,NaN,NaN,NaN,,"Clinical Fellow, Division of Emergency Medicine",[ Emergency Medicine]


In [227]:
matched, missing, duplicates = roster_match.match(cinc, ppd_spec)

1255 duplicates found.
127 names missing.
250 duplicates found.
56 unique matches found.
652
708
56 new matches confirmed. 708 total matches confirmed.
10 duplicates found.
22 unique matches found.
708
730
22 new matches confirmed. 730 total matches confirmed.
6 duplicates found.
0 unique matches found.
0 new matches confirmed. 730 total matches confirmed.

DOUGLAS

GARRETT
S.
ELIZABETH
S.
SLATTER
A.
LEE
A.
ASHER
0 duplicates found.
2 unique matches found.
2 new matches confirmed. 732 total matches confirmed.


C:\Users\vigrose\Jupyter Notebooks\Sandbox\roster_match_6.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_df['STATE_MATCH']=state_match_list
C:\Users\vigrose\Jupyter Notebooks\Sandbox\roster_match_6.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_df['CITY_MATCH']=city_match_list


In [223]:
ppd_spec['DESC_PRIM']=ppd_spec.DESC

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
for row in PPD[(PPD['FIRST_NAME']=='JAMES')&(PPD['LAST_NAME']=='WILLIAMS')].itertuples():
    print(row.MIDDLE_NAME[0])

R
E
P
G
H
W
J
H
T
R
E
M
D
E
B
R
M
P
D
R
J
R
M
B
N
K
P
E
B
H
A
F
R
L
M
D
R
M
H
S
D
E
L
T
M
M
A
R
B
E
T
M
A
O
L
R
W
D
L
T
A
B
W
G
S
R
T
M
S
C
E
L


In [66]:
matched.drop_duplicates('NAME')

,CITY_MATCH,DEGREE,DEPARTMENT,Email,FIRST_NAME,LAST_NAME,LINK,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,Link,...,MID_MATCH,NAME,NICKNAME,POLO_CITY,POLO_STATE,STATE_MATCH,SUFFIX_x,SUFFIX_y,TITLE_x,TITLE_y
0,NaN,MD\r\n,UC Department of Pediatrics/UC Department of P...,john.morehous@cchmc.org,JOHN,MOREHOUS,https://www.cincinnatichildrens.org/bio/m/joh...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/m/joh...,...,NaN,John F. Morehous,,FAIRFIELD,OH,NaN,None,,,"Medical Director, Fairfield Primary Care /Staf..."
1,NaN,MD,UC Department of Pediatrics/Associate Professo...,kelsey.logan@cchmc.org,KELSEY,LOGAN,https://www.cincinnatichildrens.org/bio/l/kel...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/l/kel...,...,NaN,Kelsey Logan,,CINCINNATI,OH,NaN,None,,,"Director, Division of Sports Medicine/Mason/Wi..."
2,NaN,MD\r\n,UC Department of Radiology/UC Department of Ra...,carl.merrow@cchmc.org,ARNOLD,MERROW,https://www.cincinnatichildrens.org/bio/m/car...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/m/car...,...,NaN,"Arnold Carlson ""Carl"" Merrow Jr.",CARL,CINCINNATI,OH,NaN,None,JR.,,"Corning Benton Chair for Radiology Education, ..."
22,NaN,MD,UC Department of Pediatrics/UC Department of P...,katherine.auger@cchmc.org,KATHERINE,AUGER,https://www.cincinnatichildrens.org/bio/a/kat...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/a/kat...,...,NaN,Katherine A. S. Auger,,CINCINNATI,OH,NaN,None,,,"Attending Physician, Division of Hospital Medi..."
23,NaN,MD,UC Department of Pediatrics/UC Department of P...,matthew.alder@cchmc.org,MATTHEW,ALDER,https://www.cincinnatichildrens.org/bio/a/mat...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/a/mat...,...,NaN,Matthew N. Alder,,CINCINNATI,OH,NaN,None,,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,True,MD | Cincinnati Children's Hospital Medical Ce...,"UC Department of Otolaryngology, Head and Neck...",matthew.smith@cchmc.org,MATTHEW,SMITH,https://www.cincinnatichildrens.org/bio/s/matt...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/s/matt...,...,True,Matthew M. Smith,,CINCINNATI,OH,True,None,,,Burnet Campus/Mason/Eastgate
662,True,MD\r\n,UC Department of Pediatrics/UC Department of P...,ryan.moore@cchmc.org,RYAN,MOORE,https://www.cincinnatichildrens.org/bio/m/rya...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/m/rya...,...,True,Ryan A. Moore,,CINCINNATI,OH,True,None,,,"Director, Heart Institute Digital Media and 3D..."
953,True,MD,None,david.katz@cchmc.org,DAVID,KATZ,https://www.cincinnatichildrens.org/bio/k/dav...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/k/dav...,...,True,David A. Katz,,CINCINNATI,OH,True,None,,,"Clinical Cardiology Fellow, Heart Institute"
1150,True,MD | Cincinnati Children's Hospital Medical Ce...,None,erin.atwood@cchmc.org,ERIN,ATWOOD,https://www.cincinnatichildrens.org/bio/a/eri...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/a/eri...,...,True,Erin S. Atwood,,CINCINNATI,OH,True,None,,,"Clinical Fellow, Division of Endocrinology/Bur..."


In [67]:
len(missing)

127

In [45]:
len(cinc)

1986

In [47]:
duplicates

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME_x,SUFFIX_x,POLO_CITY,POLO_STATE,NAME,MIDDLE_NAME_y,SUFFIX_y,NICKNAME,TITLE,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,STATE_MATCH,CITY_MATCH,MID_MATCH
179,00102781333,CHARLES M MYER III MD,MYER,CHARLES,MONROE,4,CINCINNATI,OH,Charles M. Myer III,M.,III,,,https://www.cincinnatichildrens.org/bio/m/cha...,charles.myer@cchmc.org,OH,CINCINNATI,True,True,True
180,03845060371,CHARLES M MYER IV MD,MYER,CHARLES,MONROE,5,CINCINNATI,OH,Charles M. Myer III,M.,III,,,https://www.cincinnatichildrens.org/bio/m/cha...,charles.myer@cchmc.org,OH,CINCINNATI,True,True,True
181,00102781333,CHARLES M MYER III MD,MYER,CHARLES,MONROE,4,CINCINNATI,OH,Charles M. Myer IV,M.,IV,,,https://www.cincinnatichildrens.org/bio/m/cha...,charlie.myer@cchmc.org,OH,CINCINNATI,True,True,True
182,03845060371,CHARLES M MYER IV MD,MYER,CHARLES,MONROE,5,CINCINNATI,OH,Charles M. Myer IV,M.,IV,,,https://www.cincinnatichildrens.org/bio/m/cha...,charlie.myer@cchmc.org,OH,CINCINNATI,True,True,True


In [68]:
MISSING_DF = cinc[cinc.NAME.isin(missing)==True]

In [25]:
def split_names(roster_df):
    '''Splits name column into components'''
    roster_df = roster_df.drop_duplicates()
    dict_list = []
    for row in roster_df.itertuples():
        name_parsed = HumanName(row.NAME)
        name_dict = {
            'NAME': row.NAME,
            'FIRST_NAME': name_parsed.first.upper(),
            'LAST_NAME': name_parsed.last.upper(),
            'MIDDLE_NAME': name_parsed.middle.upper(),
            'SUFFIX': name_parsed.suffix.upper(),
            'NICKNAME': name_parsed.nickname.upper(),
            'TITLE': name_parsed.title.upper(),
        }
        dict_list.append(name_dict)
    name_df = pd.DataFrame(dict_list)
    new_df = pd.merge(name_df, roster_df, on='NAME')
    return new_df

def append_me(roster_df):
    '''Matches to PPD and appends ME'''
    data_split = split_names(roster_df)
    mes = []
    for row in data_split.itertuples():
        physician_me = 'None'
        keep = True
        if keep:
            new_df = PPD[(PPD.FIRST_NAME == row.FIRST_NAME) & (PPD.LAST_NAME == row.LAST_NAME)]
            if len(new_df) == 0:
                if '-' in row.LAST_NAME:
                    last = row.LAST_NAME.replace('-', ' ')
                elif ' ' in row.LAST_NAME:
                    last = row.LAST_NAME.replace(' ', '')
                else:
                    last = row.LAST_NAME
                new_df = PPD[(PPD.LAST_NAME == last)]
                if len(new_df) == 0:
                    pass
                if len(new_df) > 1:
                    new_df = new_df[new_df.POLO_STATE == row.LOCATION_ADDRESS_STATE]
                if len(new_df) > 1:
                    new_df = new_df[new_df.POLO_CITY == row.LOCATION_ADDRESS_CITY]
            if len(new_df) == 1:
                physician_me = new_df.iloc[0]['ME']
            elif len(new_df) > 1:
                print(f'{row.NAME} potentially matched to multiple ME numbers.')
        mes.append(physician_me)
    data_split['ME'] = fix_me(mes)
    data_me = data_split[data_split.ME != 'None']
    return data_split, data_me

In [70]:
split_names, phys = append_me(MISSING_DF)

 Lynn Babcock potentially matched to multiple ME numbers.
 Sandy Durrani potentially matched to multiple ME numbers.
 Jenny Hellmann potentially matched to multiple ME numbers.
 Foong-Yen Lim potentially matched to multiple ME numbers.
 Parinda A. Mehta potentially matched to multiple ME numbers.
 Manisha Mills potentially matched to multiple ME numbers.
 Esi M. Morgan potentially matched to multiple ME numbers.
 Rupi Parikh potentially matched to multiple ME numbers.
 Felicia A. Scaggs Huang potentially matched to multiple ME numbers.
 Katie Schultz potentially matched to multiple ME numbers.
 J. Michael Taylor potentially matched to multiple ME numbers.
 Yaning Wu potentially matched to multiple ME numbers.
 Nana-Hawa Yayah Jones potentially matched to multiple ME numbers.


In [136]:
split_names[split_names.LAST_NAME=='TAYLOR']

,NAME,FIRST_NAME,LAST_NAME,MIDDLE_NAME,SUFFIX,NICKNAME,TITLE_x,DEGREE,TITLE_y,DEPARTMENT,LINK,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD,ME
107,J. Michael Taylor,J.,TAYLOR,MICHAEL,,,,MD\r\n,Pediatric Stroke and Neurocritical Care Specia...,UC Department of Pediatrics/UC Department of P...,https://www.cincinnatichildrens.org/bio/t/j-mi...,https://www.cincinnatichildrens.org/bio/t/j-mi...,jm.taylor@cchmc.org,OH,CINCINNATI,True,None


In [144]:
ppd_spec[(ppd_spec.FIRST_NAME=='NANA')&(ppd_spec.LAST_NAME=='YAYAH')]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,POLO_CITY,POLO_STATE,DESC
1098799,02002020826,NANA H YAYAH MD,YAYAH,NANA,HAWA,NaN,CINCINNATI,OH,PEDIATRIC ENDOCRINOLOGY


In [73]:
spec = pd.read_csv('../../Data/Rosters/specialty_id.csv')

In [75]:
spec.columns

Index(['SPEC_CD', 'SPEC_ID', 'DESC'], dtype='object')

In [76]:
PPD_SPEC = pd.merge(ppd, spec, left_on = 'PRIM_SPEC_CD',right_on = 'SPEC_CD')

In [79]:
ppd_spec = PPD_SPEC[['ME', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 
       'POLO_CITY', 'POLO_STATE',  'DESC']]

In [81]:
ppd_spec['ME']=fix_me(ppd_spec.ME)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [153]:
all_that = pd.concat([matched, phys[phys.ME!='None']])

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [145]:
duplicates

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME_x,SUFFIX_x,POLO_CITY,POLO_STATE,NAME,MIDDLE_NAME_y,...,DEPARTMENT,LINK,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD,STATE_MATCH,CITY_MATCH,MID_MATCH
122,00102781333,CHARLES M MYER III MD,MYER,CHARLES,MONROE,4,CINCINNATI,OH,Charles M. Myer III,M.,...,UC Department of Pediatrics/UC Department of O...,https://www.cincinnatichildrens.org/bio/m/cha...,https://www.cincinnatichildrens.org/bio/m/cha...,charles.myer@cchmc.org,OH,CINCINNATI,True,True,True,True
123,03845060371,CHARLES M MYER IV MD,MYER,CHARLES,MONROE,5,CINCINNATI,OH,Charles M. Myer III,M.,...,UC Department of Pediatrics/UC Department of O...,https://www.cincinnatichildrens.org/bio/m/cha...,https://www.cincinnatichildrens.org/bio/m/cha...,charles.myer@cchmc.org,OH,CINCINNATI,True,True,True,True
124,00102781333,CHARLES M MYER III MD,MYER,CHARLES,MONROE,4,CINCINNATI,OH,Charles M. Myer IV,M.,...,"UC Department of Otolaryngology, Head & Neck S...",https://www.cincinnatichildrens.org/bio/m/cha...,https://www.cincinnatichildrens.org/bio/m/cha...,charlie.myer@cchmc.org,OH,CINCINNATI,True,True,True,True
125,03845060371,CHARLES M MYER IV MD,MYER,CHARLES,MONROE,5,CINCINNATI,OH,Charles M. Myer IV,M.,...,"UC Department of Otolaryngology, Head & Neck S...",https://www.cincinnatichildrens.org/bio/m/cha...,https://www.cincinnatichildrens.org/bio/m/cha...,charlie.myer@cchmc.org,OH,CINCINNATI,True,True,True,True


In [147]:
all_that.columns

Index(['CITY_MATCH', 'DEGREE', 'DEPARTMENT', 'Email', 'FIRST_NAME',
       'LAST_NAME', 'LINK', 'LOCATION_ADDRESS_CITY', 'LOCATION_ADDRESS_STATE',
       'Link', 'MAILING_NAME', 'MD', 'ME', 'MIDDLE_NAME', 'MIDDLE_NAME_x',
       'MIDDLE_NAME_y', 'MID_MATCH', 'NAME', 'NICKNAME', 'POLO_CITY',
       'POLO_STATE', 'STATE_MATCH', 'SUFFIX', 'SUFFIX_x', 'SUFFIX_y',
       'TITLE_x', 'TITLE_y'],
      dtype='object')

In [158]:
pd.merge(ppd_spec, phys, on='ME').to_csv('../../Data/Rosters/Cincinnati_Pot.csv', index=False)

In [151]:
phys = phys.fillna('None')

In [159]:
pd.merge(ppd_spec, matched, on='ME').to_csv('../../Data/Rosters/Cincinnati_Match.csv', index=False)

In [160]:
matched

,CITY_MATCH,DEGREE,DEPARTMENT,Email,FIRST_NAME,LAST_NAME,LINK,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,Link,...,MID_MATCH,NAME,NICKNAME,POLO_CITY,POLO_STATE,STATE_MATCH,SUFFIX_x,SUFFIX_y,TITLE_x,TITLE_y
0,NaN,MD\r\n,UC Department of Pediatrics/UC Department of P...,john.morehous@cchmc.org,JOHN,MOREHOUS,https://www.cincinnatichildrens.org/bio/m/joh...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/m/joh...,...,NaN,John F. Morehous,,FAIRFIELD,OH,NaN,None,,,"Medical Director, Fairfield Primary Care /Staf..."
1,NaN,MD,UC Department of Pediatrics/Associate Professo...,kelsey.logan@cchmc.org,KELSEY,LOGAN,https://www.cincinnatichildrens.org/bio/l/kel...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/l/kel...,...,NaN,Kelsey Logan,,CINCINNATI,OH,NaN,None,,,"Director, Division of Sports Medicine/Mason/Wi..."
2,NaN,MD\r\n,UC Department of Radiology/UC Department of Ra...,carl.merrow@cchmc.org,ARNOLD,MERROW,https://www.cincinnatichildrens.org/bio/m/car...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/m/car...,...,NaN,"Arnold Carlson ""Carl"" Merrow Jr.",CARL,CINCINNATI,OH,NaN,None,JR.,,"Corning Benton Chair for Radiology Education, ..."
22,NaN,MD,UC Department of Pediatrics/UC Department of P...,katherine.auger@cchmc.org,KATHERINE,AUGER,https://www.cincinnatichildrens.org/bio/a/kat...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/a/kat...,...,NaN,Katherine A. S. Auger,,CINCINNATI,OH,NaN,None,,,"Attending Physician, Division of Hospital Medi..."
23,NaN,MD,UC Department of Pediatrics/UC Department of P...,matthew.alder@cchmc.org,MATTHEW,ALDER,https://www.cincinnatichildrens.org/bio/a/mat...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/a/mat...,...,NaN,Matthew N. Alder,,CINCINNATI,OH,NaN,None,,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,True,MD | Cincinnati Children's Hospital Medical Ce...,"UC Department of Otolaryngology, Head and Neck...",matthew.smith@cchmc.org,MATTHEW,SMITH,https://www.cincinnatichildrens.org/bio/s/matt...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/s/matt...,...,True,Matthew M. Smith,,CINCINNATI,OH,True,None,,,Burnet Campus/Mason/Eastgate
662,True,MD\r\n,UC Department of Pediatrics/UC Department of P...,ryan.moore@cchmc.org,RYAN,MOORE,https://www.cincinnatichildrens.org/bio/m/rya...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/m/rya...,...,True,Ryan A. Moore,,CINCINNATI,OH,True,None,,,"Director, Heart Institute Digital Media and 3D..."
953,True,MD,None,david.katz@cchmc.org,DAVID,KATZ,https://www.cincinnatichildrens.org/bio/k/dav...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/k/dav...,...,True,David A. Katz,,CINCINNATI,OH,True,None,,,"Clinical Cardiology Fellow, Heart Institute"
1150,True,MD | Cincinnati Children's Hospital Medical Ce...,None,erin.atwood@cchmc.org,ERIN,ATWOOD,https://www.cincinnatichildrens.org/bio/a/eri...,CINCINNATI,OH,https://www.cincinnatichildrens.org/bio/a/eri...,...,True,Erin S. Atwood,,CINCINNATI,OH,True,None,,,"Clinical Fellow, Division of Endocrinology/Bur..."


In [161]:
pd.merge(ppd_spec, matched, on='ME')

,ME,MAILING_NAME_x,LAST_NAME_x,FIRST_NAME_x,MIDDLE_NAME,SUFFIX,POLO_CITY_x,POLO_STATE_x,DESC,CITY_MATCH,...,MID_MATCH,NAME,NICKNAME,POLO_CITY_y,POLO_STATE_y,STATE_MATCH,SUFFIX_x,SUFFIX_y,TITLE_x,TITLE_y
0,00102831501,TRACY TILLER MD,TILLER,TRACY,NaN,NaN,FAIRFIELD,OH,EMERGENCY MEDICINE,NaN,...,NaN,Tracy Tiller,,FAIRFIELD,OH,NaN,None,,,"Staff Physician II, Division of General and Co..."
1,03501071438,WHITNEY K BRYANT MD,BRYANT,WHITNEY,K,NaN,CINCINNATI,OH,EMERGENCY MEDICINE,NaN,...,NaN,Whitney K. Bryant,,CINCINNATI,OH,NaN,None,,,None
2,00102000285,JOHN F MOREHOUS MD,MOREHOUS,JOHN,FINCH,NaN,FAIRFIELD,OH,PEDIATRICS,NaN,...,NaN,John F. Morehous,,FAIRFIELD,OH,NaN,None,,,"Medical Director, Fairfield Primary Care /Staf..."
3,00102050711,KATHERINE A AUGER MD,AUGER,KATHERINE,ANN,NaN,CINCINNATI,OH,PEDIATRICS,NaN,...,NaN,Katherine A. S. Auger,,CINCINNATI,OH,NaN,None,,,"Attending Physician, Division of Hospital Medi..."
4,00102090934,MEGAN C HENNIGAN MD,HENNIGAN,MEGAN,COLLEEN,NaN,CINCINNATI,OH,PEDIATRICS,NaN,...,NaN,Megan C. Hennigan,,CINCINNATI,OH,NaN,None,,,"Staff Physician II, CHECK Foster Care Clinic, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,02608800053,CARL L BACKER MD,BACKER,CARL,LEWIS,NaN,CHICAGO,IL,PEDIATRIC CARDIOTHORACIC SURGERY,NaN,...,NaN,Carl L. Backer,,CHICAGO,IL,NaN,None,,,Burnet Campus
650,03508991621,JASON FRISCHER MD,FRISCHER,JASON,NaN,NaN,CINCINNATI,OH,PEDIATRIC CARDIOTHORACIC SURGERY,NaN,...,NaN,Jason S. Frischer,,CINCINNATI,OH,NaN,None,,,"Director, Colorectal Center/Director, Extracor..."
651,02608060297,ANDREW T TROUT MD,TROUT,ANDREW,TIMOTHY,NaN,CINCINNATI,OH,NUCLEAR RADIOLOGY,NaN,...,NaN,Andrew T. Trout,,CINCINNATI,OH,NaN,None,,,"Director, Clinical Research, Department of Rad..."
652,00801952194,JILDA N VARGUS-ADAMS MD,VARGUS-ADAMS,JILDA,N,NaN,CINCINNATI,OH,PEDIATRICS/PHYSICAL MEDICINE AND REHABILITATION,NaN,...,NaN,Jilda N. Vargus-Adams,,CINCINNATI,OH,NaN,None,,,"Attending Physician, Division of Rehabilitatio..."


In [166]:
[x.split('ment of ') for x in cinc.DEPARTMENT]

[['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['Adjunct Associate Professor, UC Depart',
  'Anesthesiology/Adjunct Associate Professor, UC Depart',
  'Anesthesiology'],
 ['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['None'],
 ['None'],
 ['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['None'],
 ['None'],
 ['None'],
 ['UC Depart', 'Surgery/UC Depart', 'Surgery'],
 ['None'],
 ['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['None'],
 ['UC Depart', 'Pediatrics/UC Depart', 'Pediatrics'],
 ['UC Depart',
  'Pediatrics/Assistant Professor, UC Depart',
  'Ophthalmology/UC Depart',
  'Pediatrics/Assistant Professor, UC Depart',
  'Ophthalmology'],

In [172]:
cinc.DEPARTMENT[3]

'Adjunct Associate Professor, UC Department of Anesthesiology/Adjunct Associate Professor, UC Department of Anesthesiology'

In [180]:
area_list = []
for thing in cinc.DEPARTMENT:
    thing_split = thing.split('/')
    areas =''
    for dept in thing_split:
        if "ment of" in thing:
            area = thing.split('ment of')[1].split('/')[0]
            if area not in areas:
                areas += f'{area} '
    area_list.append(areas[1:-1])

In [197]:
cinc['DEPARTMENTS'] = cinc.DEPARTMENT

In [198]:
cinc['DEPARTMENT'] = area_list

In [185]:
cinc.TITLE[2]

'Medical Director, Pancreas Care Center/Pediatric Gastroenterologist, Division of Pediatric Gastroenterology/Burnet Campus'

In [728]:
dinc_do =dinc_do.fillna("None")

In [729]:
def do_thing(cinc):
    area_list = []
    for thing in cinc.DEPARTMENT:
        thing_split = thing.split('/')
        areas =''
        for dept in thing_split:
            if "ment of" in thing:
                area = thing.split('ment of')[1].split('/')[0]
                if area not in areas:
                    areas += f'{area} '
        area_list.append(areas[1:-1])
        all_titles = []
    for title in cinc.TITLE:
        title_split = title.split('/')
        title_list =[]
        for element in title_split:
            element = element.replace('Medical','').replace('Director','').replace(', ','').replace('Center','').replace('Division of','')
            element = element.replace('Program','').replace('Associate','').replace('Staff','').replace('Department of','').replace('Clinical Fellow','')
            element = element.replace('Education','').replace('Fellowship','').replace('Attending','').replace('Member','').replace('Physician','')
            title_list.append(element)
        all_titles.append(title_list)
    return(area_list, all_titles)
a_list, t_list = do_thing(dinc_do)

In [730]:
dinc_do['DEPARTMNETS'] = dinc_do['DEPARTMENT']
dinc_do['DEPARTMNET'] = a_list
dinc_do['TITLES'] = t_list

In [ ]:
bad_words = ['Township', 'Institute', 'Campus', '']

In [204]:
all_titles = []
for title in cinc.TITLE:
    title_split = title.split('/')
    title_list =[]
    for element in title_split:
        element = element.replace('Medical','').replace('Director','').replace(', ','').replace('Center','').replace('Division of','')
        element = element.replace('Program','').replace('Associate','').replace('Staff','').replace('Department of','').replace('Clinical Fellow','')
        element = element.replace('Education','').replace('Fellowship','').replace('Attending','').replace('Member','').replace('Physician','')
        title_list.append(element)
    all_titles.append(title_list)

In [205]:
all_titles

[['  ', ' Leukemia', 'Lymphoma ', 'Burnet Campus'],
 ['Burnet Campus', 'Green Township', 'Liberty Campus'],
 [' Pancreas Care ',
  'Pediatric Gastroenterologist Pediatric Gastroenterology',
  'Burnet Campus'],
 [' Anesthesiologist Anesthesia'],
 [' Neonatal-Perinatal  '],
 [' NeonatologistNeonatology and Pulmonary Biology', 'Perinatal Institute'],
 [' Neonatology', 'Burnet Campus'],
 [' Pediatric General and Thoracic Surgery'],
 ['None'],
 [' Adult Congenital Heart Disease',
  'Burnet Campus',
  "Cincinnati Children's Heart Institute – Louisville",
  "Cincinnati Children's Heart Institute – Elizabethtown"],
 [' Endocrinology', 'Burnet Campus'],
 ['  Oncology'],
 [' Neonatology and Pulmonary Biology', 'Burnet Campus'],
 ['Co-Surgical Intestinal Transplant Surgery', 'Burnet Campus'],
 ['ResidentChild Neurology ',
  ' Pediatrics Combined Residency ',
  'Burnet Campus'],
 ['Pediatric CardiologistImaging', 'Burnet Campus'],
 [' Cardiac Intensive Care Unit'],
 [' Pulmonary Medicine',
  'The 

In [206]:
cinc['TITLES'] = all_titles

In [210]:
cinc

,DEGREE,TITLE,DEPARTMENT,LINK,NAME,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD,DEPARTMENTS,TITLES
0,MD,"Director, Medical Education Program/Associate ...",Pediatrics,https://www.cincinnatichildrens.org/bio/a/mic...,Michael J. Absalon,https://www.cincinnatichildrens.org/bio/a/mic...,michael.absalon@cchmc.org,OH,CINCINNATI,True,UC Department of Pediatrics/UC Department of P...,"[ , Leukemia, Lymphoma , Burnet Campus]"
1,MD\r\n,Burnet Campus/Green Township/Liberty Campus,Pediatrics,https://www.cincinnatichildrens.org/bio/a/jua...,J. Pablo Abonia,https://www.cincinnatichildrens.org/bio/a/jua...,pablo.abonia@cchmc.org,OH,CINCINNATI,True,UC Department of Pediatrics/UC Department of P...,"[Burnet Campus, Green Township, Liberty Campus]"
2,MD\r\n,"Medical Director, Pancreas Care Center/Pediatr...",Pediatrics,https://www.cincinnatichildrens.org/bio/a/mai...,Maisam A. Abu-El-Haija,https://www.cincinnatichildrens.org/bio/a/mai...,maisam.haija@cchmc.org,OH,CINCINNATI,True,UC Department of Pediatrics/UC Department of P...,"[ Pancreas Care , Pediatric Gastroenterologist..."
3,MD\r\n,"Staff Anesthesiologist, Department of Anesthesia",Anesthesiology,https://www.cincinnatichildrens.org/bio/a/ele...,Elena Adler,https://www.cincinnatichildrens.org/bio/a/ele...,elena.adler@cchmc.org,OH,CINCINNATI,True,"Adjunct Associate Professor, UC Department of ...",[ Anesthesiologist Anesthesia]
4,MD\r\n,"Associate Director, Neonatal-Perinatal Fellows...",Pediatrics,https://www.cincinnatichildrens.org/bio/a/sha...,Shawn K. Ahlfeld,https://www.cincinnatichildrens.org/bio/a/sha...,shawn.ahlfeld@cchmc.org,OH,CINCINNATI,True,UC Department of Pediatrics/UC Department of P...,[ Neonatal-Perinatal ]
...,...,...,...,...,...,...,...,...,...,...,...,...
806,MD,"Attending Physician, Division of Critical Care...",Pediatrics,https://www.cincinnatichildrens.org/bio/z/matt...,Matthew W. Zackoff,https://www.cincinnatichildrens.org/bio/z/matt...,matthew.zackoff@cchmc.org,OH,CINCINNATI,True,UC Department of Pediatrics/UC Department of P...,[ Critical Care Medicine]
807,MD\r\n,"Member, Division of Pulmonary Medicine/Burnet ...",Pediatrics,https://www.cincinnatichildrens.org/bio/z/sara...,Sara M. Zak,https://www.cincinnatichildrens.org/bio/z/sara...,sara.zak@cchmc.org,OH,CINCINNATI,True,UC Department of Pediatrics/UC Department of P...,"[ Pulmonary Medicine, Burnet Campus]"
808,MD,Pharmacogenomics,Pediatrics,https://www.cincinnatichildrens.org/bio/z/ge-z...,Ge Zhang,https://www.cincinnatichildrens.org/bio/z/ge-z...,ge.zhang@cchmc.org,OH,CINCINNATI,True,UC Department of Pediatrics/UC Department of P...,[Pharmacogenomics]
809,MD,"Director, Laboratory of Genetics and Genomics ...",Pediatrics,https://www.cincinnatichildrens.org/bio/z/weny...,Wenying Zhang,https://www.cincinnatichildrens.org/bio/z/weny...,wenying.zhang@cchmc.org,OH,CINCINNATI,True,UC Department of Pediatrics/UC Department of P...,[Laboratory of Genetics and Genomics (LGG) Hum...


In [245]:
matched = matched.dropna(subset= ['ME'])

In [234]:
matched.groupby('ME').count().sort_values('DEGREE', ascending=False)

,CITY_MATCH,DEGREE,DEPARTMENT,DEPARTMENTS,DESC,DESC_PRIM,Email,FIRST_NAME,LAST_NAME,LINK,...,NAME,NICKNAME,POLO_CITY,POLO_STATE,SPEC_MATCH,STATE_MATCH,SUFFIX_x,SUFFIX_y,TITLE,TITLES
ME,,,,,,,,,,,,,,,,,,,,,
00102000285,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,1,1,1
04101070331,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,1,1,1
04101111780,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,1,1,1
04101151188,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,1,1,1
04101740400,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
02803090184,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,1,1,1
02803090494,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,1,1,1
02803120431,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,1,1,1


In [235]:
len(cinc)

811

In [241]:
missing

[' J. Pablo Abonia',
 ' Faris N. Al Gharaibeh',
 ' Tamador Al-Shamaileh',
 ' Sheila T. Angeles-Han',
 ' Ana Catalina (Cata) Arce-Clachar',
 ' Amal H. Assaad',
 ' Lynn Babcock',
 ' Stephanie Bachi de Castro Oliveira',
 ' Maria E. Barnes-Davis',
 ' Benoit A. Beauve',
 ' Jacob (Jack) J.H. Bleesing',
 ' Ashlee K. Bolger',
 ' Carina Braeutigam',
 ' Maria A. Calvo-Garcia',
 ' Lauren Capozza',
 ' Marguerite M. Caré',
 ' Kathleen Colman',
 ' Carolina M. Cuba Bustinza',
 ' Jennifer Cully',
 ' Stephanie Davis-Rodriguez',
 ' Peter de Blank',
 ' Nick J. DeBlasio',
 ' William R. DeFoor Jr.',
 ' Holly Depinet',
 ' Jennifer E. deSante-Bertkau',
 ' Mariko D. DeWire-Schottmiller',
 ' Thomas G. DeWitt',
 ' Allison A. Divanovic',
 ' Carolyn Dress',
 ' Sandy Durrani',
 ' Haithem M. Elhadi Babiker',
 ' Ting Ting Fu',
 ' Mike A. Gittelman',
 ' Javier A. Gonzalez del Rey',
 ' Elizabeth S. Gosnell',
 ' Qiaoning Guan',
 ' Katharine M. Guarnieri',
 ' Iris Gutmark-Little',
 ' Gretchen  Hartz',
 ' Jenny Hellmann'

In [243]:
ppd['MEDSCHOOL_STATE']

0            1
1            1
2            1
3            1
4            1
          ... 
1283761    965
1283762    965
1283763    965
1283764    965
1283765    965
Name: MEDSCHOOL_STATE, Length: 1283766, dtype: int64

In [244]:
matched

,CITY_MATCH,DEGREE,DEPARTMENT,DEPARTMENTS,DESC,DESC_PRIM,Email,FIRST_NAME,LAST_NAME,LINK,...,NAME,NICKNAME,POLO_CITY,POLO_STATE,SPEC_MATCH,STATE_MATCH,SUFFIX_x,SUFFIX_y,TITLE,TITLES
0,NaN,MD,,None,EMERGENCY MEDICINE,EMERGENCY MEDICINE,tracy.tiller@cchmc.org,TRACY,TILLER,https://www.cincinnatichildrens.org/bio/t/trac...,...,Tracy Tiller,,FAIRFIELD,OH,NaN,NaN,NaN,,"Staff Physician II, Division of General and Co...","[ II General and Community Pediatrics, Fairfi..."
493,NaN,MD,Emergency Medicine,UC Department of Emergency Medicine/UC Departm...,EMERGENCY MEDICINE,EMERGENCY MEDICINE,whitney.bryant@cchmc.org,WHITNEY,BRYANT,https://www.cincinnatichildrens.org/bio/b/whi...,...,Whitney K. Bryant,,CINCINNATI,OH,NaN,NaN,NaN,,None,[None]
717,NaN,MD\r\n,Pediatrics,UC Department of Pediatrics/UC Department of P...,PEDIATRICS,PEDIATRICS,john.morehous@cchmc.org,JOHN,MOREHOUS,https://www.cincinnatichildrens.org/bio/m/joh...,...,John F. Morehous,,FAIRFIELD,OH,NaN,NaN,NaN,,"Medical Director, Fairfield Primary Care /Staf...","[ Fairfield Primary Care , General and Commu..."
718,NaN,MD,Pediatrics,UC Department of Pediatrics/UC Department of P...,PEDIATRICS,PEDIATRICS,katherine.auger@cchmc.org,KATHERINE,AUGER,https://www.cincinnatichildrens.org/bio/a/kat...,...,Katherine A. S. Auger,,CINCINNATI,OH,NaN,NaN,NaN,,"Attending Physician, Division of Hospital Medi...","[ Hospital Medicine, Chair of Outcomes Pedia..."
719,NaN,MD\r\n,,None,PEDIATRICS,PEDIATRICS,megan.hennigan@cchmc.org,MEGAN,HENNIGAN,https://www.cincinnatichildrens.org/bio/h/meg...,...,Megan C. Hennigan,,CINCINNATI,OH,NaN,NaN,NaN,,"Staff Physician II, CHECK Foster Care Clinic, ...",[ IICHECK Foster Care Clinic General and Comm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,NaN,MD | Cincinnati Children's Hospital Medical Ce...,,None,PEDIATRIC INFECTIOUS DISEASE,PEDIATRIC INFECTIOUS DISEASE,faiza.ali@cchmc.org,FAIZA,ALI,https://www.cincinnatichildrens.org/bio/a/fai...,...,Faiza Ali,,CINCINNATI,OH,True,True,NaN,,"Staff Physician, Division of Oncology",[ Oncology]
1431,NaN,MD | Cincinnati Children's Hospital Medical Ce...,,None,PEDIATRIC HEMATOLOGY-ONCOLOGY,PEDIATRIC HEMATOLOGY-ONCOLOGY,sarah.fitzgerald@cchmc.org,SARAH,FITZGERALD,https://www.cincinnatichildrens.org/bio/f/sar...,...,Sarah Fitzgerald,,CINCINNATI,OH,True,True,NaN,,"Staff Physician, Division of Oncology",[ Oncology]
1489,NaN,MD,,None,PEDIATRIC CRITICAL CARE MEDICINE,PEDIATRIC CRITICAL CARE MEDICINE,andrea.maxwell@cchmc.org,ANDREA,MAXWELL,https://www.cincinnatichildrens.org/bio/m/and...,...,Andrea Maxwell,,CINCINNATI,OH,True,True,NaN,,"Clinical Fellow, Division of Critical Care Med...",[ Critical Care Medicine]
791,True,MD | Cincinnati Children's Hospital Medical Ce...,,None,PEDIATRIC ENDOCRINOLOGY,PEDIATRIC ENDOCRINOLOGY,erin.atwood@cchmc.org,ERIN,ATWOOD,https://www.cincinnatichildrens.org/bio/a/eri...,...,Erin S. Atwood,,CINCINNATI,OH,True,True,NaN,,"Clinical Fellow, Division of Endocrinology/Bur...","[ Endocrinology, Burnet Campus]"


In [797]:
with_em = pd.merge(ALL_MATCH, emails, how='left')

In [768]:
EMAILS['PREFERRED'] = EMAILS['PURPOSE_USG_DESC']=='Preferred E-mail'

In [769]:
emails = EMAILS.sort_values('PREFERRED').drop_duplicates(['ME','EMAIL_ADDR'], keep='last')

In [827]:
EMAILS.sort_values('PREFERRED')

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,PREFERRED
0,00102000021,merikingmartin@yahoo.com,AMA Web Site,False
2292365,04177170530,sharoncai@pcom.edu,No Usage,False
2292367,04177170530,sharoncai@pcom.edu,Single Sign-On,False
2292368,04177170548,dejharawe@pcom.edu,AMA Web Site,False
2292369,04177170548,dejharawe@pcom.edu,Morning Rounds Daily Edition ...,False
...,...,...,...,...
2928100,15401700029,trinitytwo@aol.com,Preferred E-mail,True
2928102,15401710032,johann.rothwangl@va.gov,Preferred E-mail,True
2072216,03901881579,w.surbeck@tbja.org,Preferred E-mail,True
1243990,02604071421,seth.vogel@allina.com,Preferred E-mail,True


In [798]:
with_em = with_em.fillna('None')
with_em.drop_duplicates('ME')[with_em.drop_duplicates('ME')['EMAIL_ADDR']=='None']

,DEGREE,TITLE,DEPARTMENT,LINK,NAME,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD,TITLES,ME,EMAIL_ADDR,PURPOSE_USG_DESC,PREFERRED
0,MD,"Staff Physician II, Division of General and Co...",,https://www.cincinnatichildrens.org/bio/t/trac...,Tracy Tiller,https://www.cincinnatichildrens.org/bio/t/trac...,tracy.tiller@cchmc.org,OH,CINCINNATI,True,"[ II General and Community Pediatrics, Fairfi...",00102831501,None,None,None
9,MD\r\n,"Director, NRSA General Pediatrics Research Fel...",Pediatrics,https://www.cincinnatichildrens.org/bio/c/kri...,Kristen A. Copeland,https://www.cincinnatichildrens.org/bio/c/kri...,kristen.copeland@cchmc.org,OH,CINCINNATI,True,"[NRSA General Pediatrics Research , General ...",00102991401,None,None,None
17,MD\r\n,"Staff Physician II, Emergency Department",,https://www.cincinnatichildrens.org/bio/g/ker...,Kerry E. Gorman,https://www.cincinnatichildrens.org/bio/g/ker...,kerry.gorman@cchmc.org,OH,CINCINNATI,True,[ IIEmergency Department],01642040177,None,None,None
20,MD\r\n,"Director, Center for Clinical and Translationa...",Pediatrics,https://www.cincinnatichildrens.org/bio/h/jam...,James E. Heubi,https://www.cincinnatichildrens.org/bio/h/jam...,sandra.geideman@cchmc.org,OH,CINCINNATI,True,[ for Clinical and Translational Science and T...,01720730932,None,None,None
21,MD | Cincinnati Children's Hospital Medical Ce...,"Staff Physician, Division of Oncology",,https://www.cincinnatichildrens.org/bio/s/juan...,Juan Salas,https://www.cincinnatichildrens.org/bio/s/juan...,juan.salas@cchmc.org,OH,CINCINNATI,True,[ Oncology],01803110928,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,MD\r\n,Pediatric Stroke and Neurocritical Care Specia...,Pediatrics,https://www.cincinnatichildrens.org/bio/t/j-mi...,J. Michael Taylor,https://www.cincinnatichildrens.org/bio/t/j-mi...,jm.taylor@cchmc.org,OH,CINCINNATI,True,['Pediatric Stroke and Neurocritical Care Spec...,03806070776,None,None,None
919,MD\r\n,"Director, Division of General and Thoracic Sur...",Surgery,https://www.cincinnatichildrens.org/bio/t/greg...,Greg M. Tiao,https://www.cincinnatichildrens.org/bio/t/greg...,greg.tiao@cchmc.org,OH,CINCINNATI,True,"[' General and Thoracic Surgery', 'Frederick C...",01611902901,None,None,None
923,MD,None,Surgery,https://www.cincinnatichildrens.org/bio/w/jeff...,Jeffrey Wacksman,https://www.cincinnatichildrens.org/bio/w/jeff...,pedsurology@cchmc.org,OH,CINCINNATI,True,['None'],03841700936,None,None,None
924,MD\r\n,"Pulmonary Fellow, Division of Pulmonary Medici...",None,https://www.cincinnatichildrens.org/bio/w/anto...,Antoinette Wannes Daou,https://www.cincinnatichildrens.org/bio/w/anto...,antoinette.wannesdaou@cchmc.org,OH,CINCINNATI,True,"['Pulmonary Fellow Pulmonary Medicine', 'Burne...",93501130221,None,None,None


In [801]:
not_null.drop_duplicates('ME')

,DEGREE,TITLE,DEPARTMENT,LINK,NAME,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD,TITLES,ME,EMAIL_ADDR,PURPOSE_USG_DESC,PREFERRED
1,MD,None,Emergency Medicine,https://www.cincinnatichildrens.org/bio/b/whi...,Whitney K. Bryant,https://www.cincinnatichildrens.org/bio/b/whi...,whitney.bryant@cchmc.org,OH,CINCINNATI,True,[None],03501071438,lakeshorepharmacy94@gmail.com,No Usage,False
3,MD\r\n,"Medical Director, Fairfield Primary Care /Staf...",Pediatrics,https://www.cincinnatichildrens.org/bio/m/joh...,John F. Morehous,https://www.cincinnatichildrens.org/bio/m/joh...,john.morehous@cchmc.org,OH,CINCINNATI,True,"[ Fairfield Primary Care , General and Commu...",00102000285,morehous@gmail.com,No Usage,False
4,MD,"Attending Physician, Division of Hospital Medi...",Pediatrics,https://www.cincinnatichildrens.org/bio/a/kat...,Katherine A. S. Auger,https://www.cincinnatichildrens.org/bio/a/kat...,katherine.auger@cchmc.org,OH,CINCINNATI,True,"[ Hospital Medicine, Chair of Outcomes Pedia...",00102050711,kasauger@gmail.com,SSO Email ...,False
6,MD\r\n,"Staff Physician II, CHECK Foster Care Clinic, ...",,https://www.cincinnatichildrens.org/bio/h/meg...,Megan C. Hennigan,https://www.cincinnatichildrens.org/bio/h/meg...,megan.hennigan@cchmc.org,OH,CINCINNATI,True,[ IICHECK Foster Care Clinic General and Comm...,00102090934,megan.hennigan@cchmc.org,Preferred E-mail,True
7,MD\r\n,"Staff Physician II, Neonatology and Pulmonary ...",,https://www.cincinnatichildrens.org/bio/p/mir...,Miriam Y. Peri,https://www.cincinnatichildrens.org/bio/p/mir...,miriam.peri@cchmc.org,OH,CINCINNATI,True,[ IINeonatology and Pulmonary Biology],00102931297,miriam_peri@trihealth.com,No Usage,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,MD\r\n,"Attending Physician, Division of Gastroenterol...",Pediatrics,https://www.cincinnatichildrens.org/bio/w/kait...,Kaitlin Whaley,https://www.cincinnatichildrens.org/bio/w/kait...,kaitlin.whaley@cchmc.org,OH,CINCINNATI,True,"[' GastroenterologyHepatology and Nutrition',...",02002130877,kaitlin.gibler@louisville.edu,No Usage,False
930,MD\r\n,"Director, Adolescent Medicine Fellowship/Burne...",Pediatrics,https://www.cincinnatichildrens.org/bio/w/lea-...,Lea E. Widdice,https://www.cincinnatichildrens.org/bio/w/lea-...,lea.widdice@cchmc.org,OH,CINCINNATI,True,"['Adolescent Medicine ', 'Burnet Campus']",05404000774,lea.widdice@cchmc.org,Preferred E-mail,True
931,MD\r\n,"Director, Interdisciplinary Feeding Team, FEES...",Pediatrics,https://www.cincinnatichildrens.org/bio/w/paul...,J. Paul Willging,https://www.cincinnatichildrens.org/bio/w/paul...,paul.willging@cchmc.org,OH,CINCINNATI,True,['Interdisciplinary Feeding TeamFEES Clinic an...,03841851853,paul.willging@cchmc.org,Single Sign-On,False
933,MD\r\n,Burnet Campus,Pediatrics,https://www.cincinnatichildrens.org/bio/w/pam-...,Pam Williams-Arya,https://www.cincinnatichildrens.org/bio/w/pam-...,pam.williams-arya@cchmc.org,OH,CINCINNATI,True,None,05501960421,pam.williams-arya@cchmc.org,No Usage,False


In [802]:
with_em[with_em.PREFERRED ==True].drop_duplicates('ME')

,DEGREE,TITLE,DEPARTMENT,LINK,NAME,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD,TITLES,ME,EMAIL_ADDR,PURPOSE_USG_DESC,PREFERRED
2,MD,None,Emergency Medicine,https://www.cincinnatichildrens.org/bio/b/whi...,Whitney K. Bryant,https://www.cincinnatichildrens.org/bio/b/whi...,whitney.bryant@cchmc.org,OH,CINCINNATI,True,[None],03501071438,whitney.bryant@gmail.com,Preferred E-mail,True
5,MD,"Attending Physician, Division of Hospital Medi...",Pediatrics,https://www.cincinnatichildrens.org/bio/a/kat...,Katherine A. S. Auger,https://www.cincinnatichildrens.org/bio/a/kat...,katherine.auger@cchmc.org,OH,CINCINNATI,True,"[ Hospital Medicine, Chair of Outcomes Pedia...",00102050711,katherine.auger@cchmc.org,Preferred E-mail,True
6,MD\r\n,"Staff Physician II, CHECK Foster Care Clinic, ...",,https://www.cincinnatichildrens.org/bio/h/meg...,Megan C. Hennigan,https://www.cincinnatichildrens.org/bio/h/meg...,megan.hennigan@cchmc.org,OH,CINCINNATI,True,[ IICHECK Foster Care Clinic General and Comm...,00102090934,megan.hennigan@cchmc.org,Preferred E-mail,True
8,MD\r\n,"Staff Physician II, Neonatology and Pulmonary ...",,https://www.cincinnatichildrens.org/bio/p/mir...,Miriam Y. Peri,https://www.cincinnatichildrens.org/bio/p/mir...,miriam.peri@cchmc.org,OH,CINCINNATI,True,[ IINeonatology and Pulmonary Biology],00102931297,miriam.oeri@cchmc.org,Preferred E-mail,True
10,MD,"Attending Physician, Division of General and C...",Pediatrics,https://www.cincinnatichildrens.org/bio/s/alex...,Alexandra M. Sims,https://www.cincinnatichildrens.org/bio/s/alex...,alexandra.sims@cchmc.org,OH,CINCINNATI,True,"[ General and Community Pediatrics, Burnet Ca...",01001131105,alex.sims1@gmail.com,Preferred E-mail,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,MD\r\n,"Attending Physician, Division of Gastroenterol...",Pediatrics,https://www.cincinnatichildrens.org/bio/w/kait...,Kaitlin Whaley,https://www.cincinnatichildrens.org/bio/w/kait...,kaitlin.whaley@cchmc.org,OH,CINCINNATI,True,"[' GastroenterologyHepatology and Nutrition',...",02002130877,kaitlin.m.gibler@gmail.com,Preferred E-mail,True
930,MD\r\n,"Director, Adolescent Medicine Fellowship/Burne...",Pediatrics,https://www.cincinnatichildrens.org/bio/w/lea-...,Lea E. Widdice,https://www.cincinnatichildrens.org/bio/w/lea-...,lea.widdice@cchmc.org,OH,CINCINNATI,True,"['Adolescent Medicine ', 'Burnet Campus']",05404000774,lea.widdice@cchmc.org,Preferred E-mail,True
932,MD\r\n,"Director, Interdisciplinary Feeding Team, FEES...",Pediatrics,https://www.cincinnatichildrens.org/bio/w/paul...,J. Paul Willging,https://www.cincinnatichildrens.org/bio/w/paul...,paul.willging@cchmc.org,OH,CINCINNATI,True,['Interdisciplinary Feeding TeamFEES Clinic an...,03841851853,tracypierson@aol.com,Preferred E-mail,True
934,MD\r\n,Burnet Campus,Pediatrics,https://www.cincinnatichildrens.org/bio/w/pam-...,Pam Williams-Arya,https://www.cincinnatichildrens.org/bio/w/pam-...,pam.williams-arya@cchmc.org,OH,CINCINNATI,True,None,05501960421,pamelawa@yahoo.com,Preferred E-mail,True


In [800]:
not_null= with_em[with_em.EMAIL_ADDR!='None']

In [803]:
math_list =[]
for row in not_null.itertuples():
    math = False
    if row.EMAIL_ADDR in row.Email:
        math =  True
    math_list.append(math)

In [804]:
not_null['EMAIL_MATCH']=math_list

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [807]:
not_null[not_null['EMAIL_MATCH']==True].groupby('PURPOSE_USG_DESC').count()

,DEGREE,TITLE,DEPARTMENT,LINK,NAME,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD,TITLES,ME,EMAIL_ADDR,PREFERRED,EMAIL_MATCH
PURPOSE_USG_DESC,,,,,,,,,,,,,,,
AMA Web Site,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
Morning Rounds Daily Edition,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
No Usage,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22
Preferred E-mail,109,109,109,109,109,109,109,109,109,109,109,109,109,109,109
SSO Email,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Single Sign-On,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20


In [289]:
cinc[cinc.NAME.isin(missing)].to_csv('../../Data/Rosters/Cincinnati_Missing.csv', index=False)

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,POLO_CITY,POLO_STATE,DESC,DESC_PRIM
0,00102000013,MICHAEL J STANFORD MD,STANFORD,MICHAEL,JAMES,NaN,NaN,NaN,EMERGENCY MEDICINE,EMERGENCY MEDICINE
1,00102000307,MITCHELL D BRANTLEY MD,BRANTLEY,MITCHELL,DAVID,NaN,NaN,NaN,EMERGENCY MEDICINE,EMERGENCY MEDICINE
2,00102000838,MELISSA W COSTELLO MD,COSTELLO,MELISSA,WYSONG,NaN,MOBILE,AL,EMERGENCY MEDICINE,EMERGENCY MEDICINE
3,00102001079,HITESH R PATEL MD,PATEL,HITESH,RAMAN,NaN,NaN,NaN,EMERGENCY MEDICINE,EMERGENCY MEDICINE
4,00102001257,ANDREW R EDWARDS MD,EDWARDS,ANDREW,REED,NaN,BIRMINGHAM,AL,EMERGENCY MEDICINE,EMERGENCY MEDICINE
...,...,...,...,...,...,...,...,...,...,...
1283761,94102710058,SUM T TRAN MD,TRAN,SUM,TIEN,NaN,VALENCIA,CA,PLASTIC SURGERY WITHIN THE HEAD & NECK (PLASTI...,PLASTIC SURGERY WITHIN THE HEAD & NECK (PLASTI...
1283762,03845830419,DOUGLAS A HANZEL MD,HANZEL,DOUGLAS,A,NaN,SAVANNAH,GA,SLEEP MEDICINE (ANESTHESIOLOGY),SLEEP MEDICINE (ANESTHESIOLOGY)
1283763,05404020619,IAN A MERRILL MD,MERRILL,IAN,ALLAN,NaN,SEATTLE,WA,SLEEP MEDICINE (ANESTHESIOLOGY),SLEEP MEDICINE (ANESTHESIOLOGY)
1283764,04804780063,STACIE C BERING MD,BERING,STACIE,CHERNIACK,NaN,NaN,NaN,HOSPICE & PALLIATIVE MEDICINE (OBSTETRICS & GY...,HOSPICE & PALLIATIVE MEDICINE (OBSTETRICS & GY...


In [782]:
not_null.drop_duplicates('ME')

,DEGREE,TITLE,DEPARTMENT,LINK,NAME,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD,TITLES,ME,EMAIL_ADDR,PURPOSE_USG_DESC,PREFERRED,EMAIL_MATCH
1,MD,None,Emergency Medicine,https://www.cincinnatichildrens.org/bio/b/whi...,Whitney K. Bryant,https://www.cincinnatichildrens.org/bio/b/whi...,whitney.bryant@cchmc.org,OH,CINCINNATI,True,[None],03501071438,lakeshorepharmacy94@gmail.com,No Usage,False,False
3,MD\r\n,"Medical Director, Fairfield Primary Care /Staf...",Pediatrics,https://www.cincinnatichildrens.org/bio/m/joh...,John F. Morehous,https://www.cincinnatichildrens.org/bio/m/joh...,john.morehous@cchmc.org,OH,CINCINNATI,True,"[ Fairfield Primary Care , General and Commu...",00102000285,morehous@gmail.com,No Usage,False,False
4,MD,"Attending Physician, Division of Hospital Medi...",Pediatrics,https://www.cincinnatichildrens.org/bio/a/kat...,Katherine A. S. Auger,https://www.cincinnatichildrens.org/bio/a/kat...,katherine.auger@cchmc.org,OH,CINCINNATI,True,"[ Hospital Medicine, Chair of Outcomes Pedia...",00102050711,kasauger@gmail.com,SSO Email ...,False,False
6,MD\r\n,"Staff Physician II, CHECK Foster Care Clinic, ...",,https://www.cincinnatichildrens.org/bio/h/meg...,Megan C. Hennigan,https://www.cincinnatichildrens.org/bio/h/meg...,megan.hennigan@cchmc.org,OH,CINCINNATI,True,[ IICHECK Foster Care Clinic General and Comm...,00102090934,megan.hennigan@cchmc.org,Preferred E-mail,True,True
7,MD\r\n,"Staff Physician II, Neonatology and Pulmonary ...",,https://www.cincinnatichildrens.org/bio/p/mir...,Miriam Y. Peri,https://www.cincinnatichildrens.org/bio/p/mir...,miriam.peri@cchmc.org,OH,CINCINNATI,True,[ IINeonatology and Pulmonary Biology],00102931297,miriam_peri@trihealth.com,No Usage,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,DO,"Attending Physician, Division of Hospital Medi...","UC Department of Pediatrics/Instructor, Mount ...",https://www.cincinnatichildrens.org/bio/p/shi...,Shivani J. Patel,https://www.cincinnatichildrens.org/bio/p/shi...,shivani.patel@cchmc.org,OH,CINCINNATI,False,[ Hospital Medicine],42201114661,shivani.patel229@gmail.com,Preferred E-mail,True,False
824,DO,"Clinical Fellow, Division of General & Communi...",None,https://www.cincinnatichildrens.org/bio/m/kei...,Keith J. Martin,https://www.cincinnatichildrens.org/bio/m/kei...,keith.martin@cchmc.org,OH,CINCINNATI,False,"[ General & Community Pediatrics, Fairfield]",04177150725,keith.jon.martin@gmail.com,Single Sign-On,False,False
826,DO | Cincinnati Children's Hospital Medical Ce...,"Director, Pediatric Neuroradiology Fellowship,...",UC Department of Radiology/UC Department of Ra...,https://www.cincinnatichildrens.org/bio/o/wil...,William T. O'Brien,https://www.cincinnatichildrens.org/bio/o/wil...,william.obrien@cchmc.org,OH,CINCINNATI,False,[Pediatric Neuroradiology Radiology and Imag...,04177031301,obrien_xray@yahoo.com,Preferred E-mail,True,False
827,DO,"Clinical Fellow, Division of Allergy and Immun...",None,https://www.cincinnatichildrens.org/bio/p/ste...,Steven P. Proper,https://www.cincinnatichildrens.org/bio/p/ste...,steven.proper@cchmc.org,OH,CINCINNATI,False,"[ Allergy and Immunology, Northern Kentucky]",02576150604,sproper@mcw.edu,Preferred E-mail,True,False


In [304]:
EMAILS.drop_duplicates('ME')

,ME,EMAIL_ADDR,PURPOSE_USG_DESC
0,00102000021,merikingmartin@yahoo.com,AMA Web Site
4,00102000048,nsmaisiak@hotmail.com,AMA Web Site
8,00102000056,rgala@ochsner.org,AMA Web Site
15,00102000064,drcladams@yahoo.com,AMA Web Site
23,00102000072,jhbankston@yahoo.com,AMA Web Site
...,...,...,...
3689029,96501950022,tony.waddimba@outlook.com,AMC E-mail. ...
3689039,96501960036,kamishelemaila@gmail.com,No Usage
3689040,96501960044,gmwiza@yahoo.com,AMA Web Site
3689044,96501960052,gmasiku@yahoo.com,AMA Web Site


In [302]:
EMAILS.groupby('PURPOSE_USG_DESC').count()

,ME,EMAIL_ADDR
PURPOSE_USG_DESC,,
AMA Web Site,454689,454689
AMC E-mail.,34181,34181
Event Email,3356,3356
Lyris E-mail.,49,49
MSOP Email,795,795
Membership Email.,83485,83485
Morning Rounds,92159,92159
Morning Rounds Daily Edition,460472,460472
No Usage,1142728,1142728


In [295]:
EMAILS.drop_duplicates('EMAIL_ADDR')

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,PREFERRED
0,00102000013,mikestanford@hotmail.com,AMA Web Site,False
4,00102000021,merikingmartin@yahoo.com,AMA Web Site,False
8,00102000048,nsmaisiak@hotmail.com,AMA Web Site,False
12,00102000056,rgala@ochsner.org,AMA Web Site,False
19,00102000064,drcladams@yahoo.com,AMA Web Site,False
...,...,...,...,...
4153808,96501960044,gmwiza@yahoo.com,AMA Web Site,False
4153812,96501960052,gmasiku@yahoo.com,AMA Web Site,False
4153816,96501980029,gmwandia@sdacc.com,No Usage,False
4153817,96501980029,gmwandia@yahoo.co.uk,Morning Rounds Daily Edition ...,False


In [313]:
ppd_test = ppd[['ME', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MD_DO_CODE', 'BIRTH_YEAR',
       'GENDER',
       'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MEDSCHOOL_STATE', 'MEDSCHOOL_GRAD_YEAR', 'POLO_CITY', 'POLO_STATE']]

In [329]:
PPD_TEST = pd.merge(ppd_test, spec, left_on='PRIM_SPEC_CD', right_on='SPEC_CD', how='left')
PPD_TEST = pd.merge(PPD_TEST, spec, left_on='SEC_SPEC_CD', right_on='SPEC_CD', suffixes = {'_PRIM', '_SEC'}, how='left')

In [320]:
PPD_TEST.dtypes

ME                       int64
MAILING_NAME            object
LAST_NAME               object
FIRST_NAME              object
MIDDLE_NAME             object
SUFFIX                 float64
MD_DO_CODE               int64
BIRTH_YEAR             float64
GENDER                 float64
PRIM_SPEC_CD            object
SEC_SPEC_CD             object
MEDSCHOOL_STATE          int64
MEDSCHOOL_GRAD_YEAR      int64
POLO_CITY               object
POLO_STATE              object
SPEC_CD_SEC             object
SPEC_ID_SEC              int64
DESC_SEC                object
SPEC_CD_PRIM            object
SPEC_ID_PRIM             int64
DESC_PRIM               object
dtype: object

In [327]:
pd.read_csv('../../Data/Rosters/STATE.csv')

,SRC_STATE_CD,STATE_ID,DESC,SRC_SYS,INSERT_TMSTP,UPDATE_TMSTP,INSERT_USERID,UPDATE_USERID,AMA_STATE_CD
0,~,-1,NOT AVAILABLE / UNKNOWN,MANUAL,13AUG2015:00:00:00.000000,13AUG2015:00:00:00.000000,MANUAL,MANUAL,NaN
1,AK,195,Alaska,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,2.0
2,AL,196,Alabama,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,1.0
3,CT,197,Connecticut,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,8.0
4,FL,198,Florida,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,11.0
...,...,...,...,...,...,...,...,...,...
60,CZ,254,Canal Zone,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,NaN
61,DE,255,Delaware,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,9.0
62,NJ,256,New Jersey,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,33.0
63,PA,257,Pennsylvania,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,41.0


In [326]:
pd.merge(STATE, PPD_TEST, left_on='STATE_ID', right_on='MEDSCHOOL_STATE')

,SRC_STATE_CD_x,STATE_ID,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,...,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM,SRC_STATE_CD_y
0,FL,198,19801610024,VASSIL V TCHOKOEV MD,TCHOKOEV,VASSIL,VASTANOV,NaN,1,1934.0,...,1961,NaN,NaN,N,1748,NEUROLOGY,FM,1691,FAMILY MEDICINE,198
1,FL,198,19801910109,LARISSA M DIMITROV MD,DIMITROV,LARISSA,M.,NaN,1,1965.0,...,1991,INDIANAPOLIS,IN,HOS,1711,HOSPITALIST,FM,1691,FAMILY MEDICINE,198
2,FL,198,19801530021,DANTON M PAPURKOV MD,PAPURKOV,DANTON,MARINOV,NaN,1,1926.0,...,1953,PITTSFIELD,MA,EM,1682,EMERGENCY MEDICINE,US,1883,UNSPECIFIED,198
3,FL,198,19801850092,CHRISTO A HADJIEV MD,HADJIEV,CHRISTO,A,NaN,1,1960.0,...,1985,NaN,NaN,EM,1682,EMERGENCY MEDICINE,US,1883,UNSPECIFIED,198
4,FL,198,19801940016,RADOSVETA N WELLS MD,WELLS,RADOSVETA,N,NaN,1,1969.0,...,1994,EL PASO,TX,EM,1682,EMERGENCY MEDICINE,US,1883,UNSPECIFIED,198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12808,NC,244,24402720014,SOU-LIN H LEE MD,LEE,SOU-LIN,H WU,NaN,1,1947.0,...,1972,RIVERSIDE,CA,R,1853,RADIOLOGY,NR,1761,NUCLEAR RADIOLOGY,244
12809,NC,244,24401790078,CHARLES C TUEN MD,TUEN,CHARLES,C,NaN,1,1953.0,...,1979,DALLAS,TX,N,1748,NEUROLOGY,CN,1665,CLINICAL NEUROPHYSIOLOGY,244
12810,NC,244,24402870067,JAMES W CHEN MD,CHEN,JAMES,WAY-YOUNG,NaN,1,1962.0,...,1987,LOS ANGELES,CA,N,1748,NEUROLOGY,CN,1665,CLINICAL NEUROPHYSIOLOGY,244
12811,NC,244,24402930043,CHAO-WEN CHEN MD,CHEN,CHAO-WEN,NaN,NaN,1,1966.0,...,1993,TALLAHASSEE,FL,N,1748,NEUROLOGY,CN,1665,CLINICAL NEUROPHYSIOLOGY,244


In [325]:
ppd_test.MEDSCHOOL_STATE

0            1
1            1
2            1
3            1
4            1
          ... 
1283761    965
1283762    965
1283763    965
1283764    965
1283765    965
Name: MEDSCHOOL_STATE, Length: 1283766, dtype: int64

In [328]:
PPD_TEST.groupby('MEDSCHOOL_STATE').count()

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,GENDER,PRIM_SPEC_CD,...,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM,SRC_STATE_CD
MEDSCHOOL_STATE,,,,,,,,,,,,,,,,,,,,,
1,11082,11082,11082,11082,10359,1053,11082,11079,11081,11082,...,11082,8834,8833,11082,11082,11082,11082,11082,11082,11082
3,7232,7232,7231,7232,5733,127,7232,6828,6875,7232,...,7232,5650,5650,7232,7232,7232,7232,7232,7232,7232
4,7304,7304,7304,7304,6894,527,7304,7304,7304,7304,...,7304,5594,5594,7304,7304,7304,7304,7304,7304,7304
5,60985,60985,60983,60985,50848,1377,60985,60734,60864,60985,...,60985,46394,46391,60985,60985,60985,60985,60985,60985,60985
7,8070,8070,8069,8070,7066,205,8070,8066,8070,8070,...,8070,5987,5986,8070,8070,8070,8070,8070,8070,8070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,14,14,14,14,12,0,14,14,14,14,...,14,8,8,14,14,14,14,14,14,14
950,51,51,51,51,44,0,51,51,51,51,...,51,28,28,51,51,51,51,51,51,51
951,35,35,35,35,35,1,35,35,35,35,...,35,26,26,35,35,35,35,35,35,35


In [721]:
first = 'LINDSEY'
last = 'BARRICK'
middle = 'MICHAEL'

In [722]:
PPD_TEST[(PPD_TEST.FIRST_NAME == first)&(PPD_TEST.LAST_NAME ==last)]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,GENDER,PRIM_SPEC_CD,...,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM
685812,3875100628,LINDSEY S BARRICK DO,BARRICK,LINDSEY,SUZANNE,NaN,2,1981.0,2.0,PEM,...,38,2010,COLUMBUS,OH,PEM,1813,PEDIATRIC EMERGENCY MEDICINE (PEDIATRICS),US,1883,UNSPECIFIED


In [509]:
PPD_TEST[(PPD_TEST.LAST_NAME == last)&(PPD_TEST.FIRST_NAME == 'KATIE')]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,GENDER,PRIM_SPEC_CD,...,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM


In [423]:
PPD_TEST[(PPD_TEST.LAST_NAME == last)]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,GENDER,PRIM_SPEC_CD,...,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM
101753,802740077,EUGENE DE BLASIO MD,DE BLASIO,EUGENE,NaN,NaN,1,1948.0,1.0,PD,...,8,1974,BAYSIDE,NY,PD,1800,PEDIATRICS,A,1628,ALLERGY
458815,2878030322,DAVID C DE BLASIO DO,DE BLASIO,DAVID,C,NaN,2,1975.0,1.0,IM,...,28,2003,FORT BRAGG,NC,IM,1724,INTERNAL MEDICINE,US,1883,UNSPECIFIED
693757,3901831784,DANL S DE BLASIO MD,DE BLASIO,DANL,SERAFINO,NaN,1,1956.0,1.0,RO,...,39,1983,WATERTOWN,NY,RO,1860,RADIATION ONCOLOGY,US,1883,UNSPECIFIED
1059127,42201831509,JOSEPH DE BLASIO MD,DE BLASIO,JOSEPH,NaN,NaN,1,1954.0,1.0,IM,...,422,1983,TRENTON,NJ,IM,1724,INTERNAL MEDICINE,US,1883,UNSPECIFIED
1059438,42201851534,THOMAS F DE BLASIO MD,DE BLASIO,THOMAS,FRANK,NaN,1,1957.0,1.0,IM,...,422,1985,MANALAPAN,NJ,IM,1724,INTERNAL MEDICINE,NTR,1765,NUTRITION
1144672,56101760125,PETER F DE BLASIO JR MD,DE BLASIO,PETER,F,2.0,1,1945.0,1.0,OPH,...,561,1976,N PROVIDENCE,RI,OPH,1783,OPHTHALMOLOGY,US,1883,UNSPECIFIED
1145801,56110670015,MARIA P DE BLASIO MD,DE BLASIO,MARIA,PIA,NaN,1,1940.0,2.0,FM,...,561,1967,BRONX,NY,FM,1691,FAMILY MEDICINE,EM,1682,EMERGENCY MEDICINE


In [433]:
PPD_TEST[(PPD_TEST.FIRST_NAME == first)]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,GENDER,PRIM_SPEC_CD,...,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM
25917,502020889,MARIKO BIRD MD,BIRD,MARIKO,NaN,NaN,1,1974.0,2.0,AN,...,5,2003,PLEASANTON,CA,AN,1639,ANESTHESIOLOGY,US,1883,UNSPECIFIED
43416,511110301,MARIKO L HOWE MD,HOWE,MARIKO,LYNNE,NaN,1,1984.0,2.0,CHN,...,5,2018,NaN,NaN,CHN,1659,CHILD NEUROLOGY,US,1883,UNSPECIFIED
81549,576100337,MARIKO A FERRONATO DO,FERRONATO,MARIKO,AILI,NaN,2,1984.0,2.0,GS,...,5,2011,SALEM,OR,GS,1703,GENERAL SURGERY,US,1883,UNSPECIFIED
85709,577120862,MARIKO ISHIYAMA DO,ISHIYAMA,MARIKO,NaN,NaN,2,1974.0,2.0,FM,...,5,2012,MODESTO,CA,FM,1691,FAMILY MEDICINE,US,1883,UNSPECIFIED
105070,1001070084,MARIKO L MUI MD,MUI,MARIKO,LINDSAY,NaN,1,1978.0,2.0,PD,...,10,2007,NaN,NaN,PD,1800,PEDIATRICS,US,1883,UNSPECIFIED
176732,1401170631,MARIKO SHIMIZU MD,SHIMIZU,MARIKO,NaN,NaN,1,1987.0,2.0,P,...,14,2017,NaN,NaN,P,1793,PSYCHIATRY,US,1883,UNSPECIFIED
191434,1606102034,MARIKO M FITZGIBBONS MD,FITZGIBBONS,MARIKO,MIYAMOTO,NaN,1,1984.0,2.0,RNR,...,16,2010,DUARTE,CA,RNR,1859,NEURORADIOLOGY,US,1883,UNSPECIFIED
198598,1606940749,MARIKO KITA MD,KITA,MARIKO,NaN,NaN,1,1968.0,2.0,N,...,16,1994,SEATTLE,WA,N,1748,NEUROLOGY,US,1883,UNSPECIFIED
203889,1611143016,MARIKO LIMPAR MD,LIMPAR,MARIKO,NaN,NaN,1,1985.0,2.0,FM,...,16,2014,GRIFFITH,IN,FM,1691,FAMILY MEDICINE,US,1883,UNSPECIFIED
226350,1643120387,MARIKO KITA MD,KITA,MARIKO,NaN,NaN,1,1981.0,2.0,OBG,...,16,2012,BLOOMFLD HLS,MI,OBG,1769,OBSTETRICS & GYNECOLOGY,US,1883,UNSPECIFIED


In [678]:
PPD_TEST[(PPD_TEST.MIDDLE_NAME == middle)&(PPD_TEST.LAST_NAME == last)&(PPD_TEST.MEDSCHOOL_GRAD_YEAR == 2008)]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,GENDER,PRIM_SPEC_CD,...,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM
643269,3806070776,JOHN M TAYLOR MD,TAYLOR,JOHN,MICHAEL,NaN,1,1980.0,1.0,CHN,...,38,2008,WASHINGTON,DC,CHN,1659,CHILD NEUROLOGY,US,1883,UNSPECIFIED


In [666]:
PPD_TEST[(PPD_TEST.LAST_NAME == 'STRIFE')&(PPD_TEST.MEDSCHOOL_GRAD_YEAR == 1968)]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,GENDER,PRIM_SPEC_CD,...,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM
498517,3305680442,JANET R STRIFE MD,STRIFE,JANET,R LANG,NaN,1,1942.0,2.0,PDR,...,33,1968,CINCINNATI,OH,PDR,1809,PEDIATRIC RADIOLOGY,CD,1655,CARDIOVASCULAR DISEASE
498542,3305680809,CYRIL F STRIFE MD,STRIFE,CYRIL,FREDERIC,NaN,1,1942.0,1.0,NEP,...,33,1968,NaN,NaN,NEP,1755,NEPHROLOGY,PD,1800,PEDIATRICS


In [684]:
PPD_TEST[(PPD_TEST.FIRST_NAME == 'KELLI')]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,GENDER,PRIM_SPEC_CD,...,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM
242,102010914,KELLI J FOLGMAN MD,FOLGMAN,KELLI,JENNIFER,NaN,1,1974.0,2.0,IM,...,1,2001,FULTONDALE,AL,IM,1724,INTERNAL MEDICINE,US,1883,UNSPECIFIED
808,102050185,KELLI H TAPLEY MD,TAPLEY,KELLI,HARNS,NaN,1,1977.0,2.0,PD,...,1,2005,BIRMINGHAM,AL,PD,1800,PEDIATRICS,US,1883,UNSPECIFIED
1514,102091116,KELLI R STRINGER MD,STRINGER,KELLI,RENAE,NaN,1,1981.0,2.0,PD,...,1,2009,HUNTSVILLE,AL,PD,1800,PEDIATRICS,US,1883,UNSPECIFIED
12791,301150245,KELLI R HICKLE MD,HICKLE,KELLI,RENEE,NaN,1,1989.0,2.0,PSI,...,3,2015,WORCESTER,MA,PSI,1922,PLASTIC SURGERY-INTEGRATED,US,1883,UNSPECIFIED
17794,376140733,KELLI HARPER DO,HARPER,KELLI,NaN,NaN,2,NaN,NaN,US,...,3,2014,MESA,AZ,US,1883,UNSPECIFIED,US,1883,UNSPECIFIED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018504,30501100923,KELLI A OGLESBY MD,OGLESBY,KELLI,ANN,NaN,1,1980.0,2.0,FM,...,305,2010,AUSTIN,TX,FM,1691,FAMILY MEDICINE,US,1883,UNSPECIFIED
1060627,42201961267,KELLI A NAYLOR MD,NAYLOR,KELLI,ANN,NaN,1,1968.0,2.0,MPD,...,422,1996,KINGSTON,RI,MPD,1743,INTERNAL MEDICINE/PEDIATRICS,US,1883,UNSPECIFIED
1179341,65401120216,KELLI M BOELENS MD,BOELENS,KELLI,M,NaN,1,1985.0,2.0,PD,...,654,2012,BLOOMSDALE,MO,PD,1800,PEDIATRICS,US,1883,UNSPECIFIED
1185501,66106130192,KELLI M GEVAS MD,GEVAS,KELLI,MARIE,NaN,1,1962.0,2.0,OBG,...,661,2013,COLUMBUS,OH,OBG,1769,OBSTETRICS & GYNECOLOGY,US,1883,UNSPECIFIED


In [706]:
PPD_TEST[(PPD_TEST.LAST_NAME == "WIDDICE")]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,BIRTH_YEAR,GENDER,PRIM_SPEC_CD,...,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_CITY,POLO_STATE,SPEC_CD_SEC,SPEC_ID_SEC,DESC_SEC,SPEC_CD_PRIM,SPEC_ID_PRIM,DESC_PRIM
923095,5404000774,ELEANOR A WIDDICE MD,WIDDICE,ELEANOR,AUSTIN,NaN,1,1970.0,2.0,ADL,...,54,2000,CINCINNATI,OH,ADL,1630,ADOLESCENT MEDICINE (PEDIATRICS),US,1883,UNSPECIFIED


In [735]:
missing_2

[' Alexis Benscoter',
 ' Emilia Connolly',
 ' Clare C. Crosh',
 ' M. Christa Krupski',
 ' Tanya Perry',
 'Natasha Pillay Smiley',
 ' Amelle L. Shillington']

In [737]:
len(matched_2)

29

In [738]:
duplicates_2

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME_x,SUFFIX_x,POLO_CITY,POLO_STATE,DESC,DESC_PRIM,...,LOCATION_ADDRESS_CITY,MD,DO,DEPARTMNETS,DEPARTMNET,TITLES,SPEC_MATCH,STATE_MATCH,CITY_MATCH,MID_MATCH


In [741]:
matched_2.sort_values('LAST_NAME')

,CITY_MATCH,DEGREE,DEPARTMENT,DEPARTMNET,DEPARTMNETS,DESC,DESC_PRIM,DO,Email,FIRST_NAME,...,NAME,NICKNAME,POLO_CITY,POLO_STATE,SPEC_MATCH,STATE_MATCH,SUFFIX_x,SUFFIX_y,TITLE,TITLES
158,NaN,DO,None,,None,PEDIATRIC GASTROENTEROLOGY,PEDIATRIC GASTROENTEROLOGY,True,gracielle.bahia@cchmc.org,GRACIELLE,...,Gracielle N. Bahia,,BUFFALO,NY,NaN,NaN,NaN,,"Clinical Fellow, Neurogastroenterology and Mot...",[Neurogastroenterology and Motility Disorder ]
153,NaN,DO,UC Department of Pediatrics/UC Department of P...,Pediatrics,UC Department of Pediatrics/UC Department of P...,PEDIATRIC EMERGENCY MEDICINE (PEDIATRICS),PEDIATRIC EMERGENCY MEDICINE (PEDIATRICS),True,lindsey.barrick@cchmc.org,LINDSEY,...,Lindsey Barrick,,COLUMBUS,OH,NaN,NaN,NaN,,None,[None]
160,NaN,DO\r\n,UC Department of Pediatrics/UC Department of P...,Pediatrics,UC Department of Pediatrics/UC Department of P...,NaN,NaN,True,alexis.ramby@cchmc.org,ALEXIS,...,Alexis Benscoter,,NaN,NaN,NaN,NaN,NaN,,"Attending Physician, Cardiac Intensive Care Un...","[ Cardiac Intensive Care Unit, Transposition o..."
143,NaN,DO\r\n,UC Department of Pediatrics/UC Department of P...,Pediatrics,UC Department of Pediatrics/UC Department of P...,SPORTS MEDICINE (PEDIATRICS),SPORTS MEDICINE (PEDIATRICS),True,kate.berz@cchmc.org,KATE,...,Kate E. Berz,,CINCINNATI,OH,NaN,NaN,NaN,,"Assistant Program Director, Pediatric Sports M...","[Assistant Pediatric Sports Medicine , Emer..."
161,NaN,DO | Cincinnati Children's Hospital Medical Ce...,UC Department of Pediatrics/UC Department of P...,Pediatrics,UC Department of Pediatrics/UC Department of P...,NaN,NaN,True,emilia.connolly@cchmc.org,EMILIA,...,Emilia Connolly,,NaN,NaN,NaN,NaN,NaN,,"Attending Physician, Division of Hospital Medi...",[ Hospital Medicine]
162,NaN,DO | Cincinnati Children's Hospital Medical Ce...,None,,None,NaN,NaN,True,clare.crosh@cchmc.org,CLARE,...,Clare C. Crosh,,NaN,NaN,NaN,NaN,NaN,,"Clinical Fellow, Division of General and Commu...","[ General and Community Pediatrics, Burnet Cam..."
154,NaN,DO,UC Department of Anesthesia/UC Department of A...,Anesthesia,UC Department of Anesthesia/UC Department of A...,HOSPICE & PALLIATIVE MEDICINE,HOSPICE & PALLIATIVE MEDICINE,True,hilary.flint@cchmc.org,HILARY,...,Hilary A. Flint,,CINCINNATI,OH,NaN,NaN,NaN,,"Pediatrician, Palliative Medicine Specialist, ...",[PediatricianPalliative Medicine Specialist An...
150,NaN,DO\r\n,UC Department of Anesthesia/UC Department of A...,Anesthesia,UC Department of Anesthesia/UC Department of A...,RHEUMATOLOGY,RHEUMATOLOGY,True,lisa.francis@cchmc.org,LISA,...,Lisa J. Francis,,NaN,NaN,NaN,NaN,NaN,,"Attending Anesthesiologist, Department of Anes...","[ Anesthesiologist Anesthesia, Intraoperative ..."
156,NaN,DO\r\n,UC Department of Pediatrics/UC Department of P...,Pediatrics,UC Department of Pediatrics/UC Department of P...,SLEEP MEDICINE,SLEEP MEDICINE,True,sleep@cchmc.org,NEEPA,...,Neepa S. Gurbani,,CINCINNATI,OH,NaN,NaN,NaN,,"Member, Division of Pulmonary Medicine/Burnet ...","[ Pulmonary Medicine, Burnet Campus, Liberty C..."
149,NaN,DO | Cincinnati Children's Hospital Medical Ce...,None,,None,PEDIATRIC HEMATOLOGY-ONCOLOGY,PEDIATRIC HEMATOLOGY-ONCOLOGY,True,laquita.jones@cchmc.org,LAQUITA,...,LaQuita Jones,,CINCINNATI,OH,NaN,NaN,NaN,,"Member, Division of Oncology",[ Oncology]


In [742]:
cinc.groupby('DEGREE').count()

,TITLE,DEPARTMENT,LINK,NAME,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD
DEGREE,,,,,,,,,
DMD,3,3,3,3,3,3,3,3,3
DMD\r\n,1,0,1,1,1,1,1,1,1
MD,243,230,258,258,258,258,258,258,258
MD\r\n,373,308,386,386,386,386,386,386,386
MD \r\n,3,1,3,3,3,3,3,3,3
MD | Cincinnati Children's Hospital Medical Center\r\n,149,63,154,154,154,154,154,154,154
MD | Louisville,1,1,1,1,1,1,1,1,1
MDiv,4,0,4,4,4,4,4,4,4
MDiv\r\n,1,0,1,1,1,1,1,1,1


In [743]:
keeps =[]
for row in cinc.itertuples():
    keep = True
    if 'DMD' in row.DEGREE or 'MDiv' in row.DEGREE:
        keep=False
    keeps.append(keep)
        
cinc['KEEP'] = keeps

In [745]:
cinc = cinc[cinc.KEEP!=False]

In [746]:
dinc_do.columns

Index(['DEGREE', 'TITLE', 'DEPARTMENT', 'LINK', 'NAME', 'Link', 'Email',
       'LOCATION_ADDRESS_STATE', 'LOCATION_ADDRESS_CITY', 'MD', 'DO',
       'DEPARTMNETS', 'DEPARTMNET', 'TITLES'],
      dtype='object')

In [751]:
cinc = cinc.drop(columns= 'KEEP')

In [753]:
xx = pd.concat([cinc, dinc_do[cinc.columns]])

In [754]:
len(xx)

840

In [756]:
xx[xx.MD==False]

,DEGREE,TITLE,DEPARTMENT,LINK,NAME,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD
0,DO,"Clinical Fellow, Neurogastroenterology and Mot...",None,https://www.cincinnatichildrens.org/bio/b/gra...,Gracielle N. Bahia,https://www.cincinnatichildrens.org/bio/b/gra...,gracielle.bahia@cchmc.org,OH,CINCINNATI,False
1,DO,None,UC Department of Pediatrics/UC Department of P...,https://www.cincinnatichildrens.org/bio/b/lin...,Lindsey Barrick,https://www.cincinnatichildrens.org/bio/b/lin...,lindsey.barrick@cchmc.org,OH,CINCINNATI,False
2,DO\r\n,"Attending Physician, Cardiac Intensive Care Un...",UC Department of Pediatrics/UC Department of P...,https://www.cincinnatichildrens.org/bio/b/ale...,Alexis Benscoter,https://www.cincinnatichildrens.org/bio/b/ale...,alexis.ramby@cchmc.org,OH,CINCINNATI,False
3,DO\r\n,"Assistant Program Director, Pediatric Sports M...",UC Department of Pediatrics/UC Department of P...,https://www.cincinnatichildrens.org/bio/b/kat...,Kate E. Berz,https://www.cincinnatichildrens.org/bio/b/kat...,kate.berz@cchmc.org,OH,CINCINNATI,False
4,DO | Cincinnati Children's Hospital Medical Ce...,"Attending Physician, Division of Hospital Medi...",UC Department of Pediatrics/UC Department of P...,https://www.cincinnatichildrens.org/bio/c/emi...,Emilia Connolly,https://www.cincinnatichildrens.org/bio/c/emi...,emilia.connolly@cchmc.org,OH,CINCINNATI,False
5,DO | Cincinnati Children's Hospital Medical Ce...,"Clinical Fellow, Division of General and Commu...",None,https://www.cincinnatichildrens.org/bio/c/cla...,Clare C. Crosh,https://www.cincinnatichildrens.org/bio/c/cla...,clare.crosh@cchmc.org,OH,CINCINNATI,False
6,DO,"Pediatrician, Palliative Medicine Specialist, ...",UC Department of Anesthesia/UC Department of A...,https://www.cincinnatichildrens.org/bio/f/hil...,Hilary A. Flint,https://www.cincinnatichildrens.org/bio/f/hil...,hilary.flint@cchmc.org,OH,CINCINNATI,False
7,DO\r\n,"Attending Anesthesiologist, Department of Anes...",UC Department of Anesthesia/UC Department of A...,https://www.cincinnatichildrens.org/bio/f/lis...,Lisa J. Francis,https://www.cincinnatichildrens.org/bio/f/lis...,lisa.francis@cchmc.org,OH,CINCINNATI,False
8,DO | Cincinnati Children's Hospital Medical Ce...,"Staff Physician, Division of Bone Marrow Trans...",None,https://www.cincinnatichildrens.org/bio/g/ash...,Ashley Green,https://www.cincinnatichildrens.org/bio/g/ash...,ashley.green@cchmc.org,OH,CINCINNATI,False
9,DO\r\n,"Member, Division of Pulmonary Medicine/Burnet ...",UC Department of Pediatrics/UC Department of P...,https://www.cincinnatichildrens.org/bio/g/nee...,Neepa S. Gurbani,https://www.cincinnatichildrens.org/bio/g/nee...,sleep@cchmc.org,OH,CINCINNATI,False


In [759]:
matched.columns

Index(['CITY_MATCH', 'DEGREE', 'DEPARTMENT', 'DEPARTMENTS', 'DESC',
       'DESC_PRIM', 'Email', 'FIRST_NAME', 'LAST_NAME', 'LINK',
       'LOCATION_ADDRESS_CITY', 'LOCATION_ADDRESS_STATE', 'Link',
       'MAILING_NAME', 'MD', 'ME', 'MIDDLE_NAME_x', 'MIDDLE_NAME_y',
       'MID_MATCH', 'NAME', 'NICKNAME', 'POLO_CITY', 'POLO_STATE',
       'SPEC_MATCH', 'STATE_MATCH', 'SUFFIX_x', 'SUFFIX_y', 'TITLE', 'TITLES'],
      dtype='object')

In [758]:
matched_2.columns

Index(['CITY_MATCH', 'DEGREE', 'DEPARTMENT', 'DEPARTMNET', 'DEPARTMNETS',
       'DESC', 'DESC_PRIM', 'DO', 'Email', 'FIRST_NAME', 'LAST_NAME', 'LINK',
       'LOCATION_ADDRESS_CITY', 'LOCATION_ADDRESS_STATE', 'Link',
       'MAILING_NAME', 'MD', 'ME', 'MIDDLE_NAME_x', 'MIDDLE_NAME_y',
       'MID_MATCH', 'NAME', 'NICKNAME', 'POLO_CITY', 'POLO_STATE',
       'SPEC_MATCH', 'STATE_MATCH', 'SUFFIX_x', 'SUFFIX_y', 'TITLE', 'TITLES'],
      dtype='object')

In [789]:
moar = pd.read_csv('../../Data/Rosters/Cincinnati_Found.csv')

In [794]:
moar = moar.drop(columns='DEPARTMENTS')

In [795]:
ALL_MATCH = pd.concat([matched[moar.columns], matched_2[moar.columns], moar])

In [791]:
moar['ME']=fix_me(moar.ME)

In [790]:
moar = moar.fillna('None')
moar = moar[moar.ME!='None']

In [796]:
len(ALL_MATCH)

720

In [792]:
len(moar)

86

In [826]:
with_em[['LINK', 'NAME',  'ME',
       'Email','EMAIL_ADDR', 'PREFERRED', 'EMAIL_MATCH']].to_excel('../../Data/Rosters/Cincinnati_Childrens_Roster_Emails.xlsx', index=False)

In [821]:
not_null.sort_values('PREFERRED').drop_duplicates('ME', keep='last')[['LINK', 'NAME', 'Email', 'ME',
       'EMAIL_ADDR', 'PREFERRED', 'EMAIL_MATCH']]

,LINK,NAME,Email,ME,EMAIL_ADDR,PREFERRED,EMAIL_MATCH
735,https://www.cincinnatichildrens.org/bio/l/jam...,James L. Leach,james.leach@cchmc.org,03841901745,leachjl2112@gmail.com,False,False
738,https://www.cincinnatichildrens.org/bio/j/vir...,Viral V. Jain,viral.jain@cchmc.org,49522990182,drviraljain@hotmail.com,False,False
338,https://www.cincinnatichildrens.org/bio/t/jame...,James S. Tweddell,james.tweddell@cchmc.org,03841851675,james.tweddell@cchmc.org,False,True
323,https://www.cincinnatichildrens.org/bio/d/les...,Lesley A. Doughty,lesley.doughty@cchmc.org,03201880211,l.doughty@cincinnatichildrens.org,False,False
740,https://www.cincinnatichildrens.org/bio/r/mich...,Michael J. Rutter,mike.rutter@cchmc.org,67102870066,michael.rutter@uc.edu,False,False
...,...,...,...,...,...,...,...
407,https://www.cincinnatichildrens.org/bio/s/kris...,Kristin Schneider,kristin.schneider@cchmc.org,04813141464,kristinschneider@gmail.com,True,False
408,https://www.cincinnatichildrens.org/bio/s/davi...,David S. Spar,david.spar@cchmc.org,05002050708,david.spar@gmail.com,True,False
409,https://www.cincinnatichildrens.org/bio/t/hieu-ta,Hieu Ta,hieu.ta@cchmc.org,05104141228,taht@vcu.edu,True,False
400,https://www.cincinnatichildrens.org/bio/v/chet...,Chet R. Villa,chet.villa@cchmc.org,03841060325,chet.villa@gmail.com,True,False


In [822]:
math_list =[]
for row in with_em.itertuples():
    math = False
    if row.EMAIL_ADDR == 'None':
        math = 'None'
    elif row.EMAIL_ADDR in row.Email:
        math =  True
    math_list.append(math)

In [823]:
with_em['EMAIL_MATCH']=math_list

In [824]:
with_em

,DEGREE,TITLE,DEPARTMENT,LINK,NAME,Link,Email,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_CITY,MD,TITLES,ME,EMAIL_ADDR,PURPOSE_USG_DESC,PREFERRED,EMAIL_MATCH
0,MD,"Staff Physician II, Division of General and Co...",,https://www.cincinnatichildrens.org/bio/t/trac...,Tracy Tiller,https://www.cincinnatichildrens.org/bio/t/trac...,tracy.tiller@cchmc.org,OH,CINCINNATI,True,"[ II General and Community Pediatrics, Fairfi...",00102831501,None,None,None,None
1,MD,None,Emergency Medicine,https://www.cincinnatichildrens.org/bio/b/whi...,Whitney K. Bryant,https://www.cincinnatichildrens.org/bio/b/whi...,whitney.bryant@cchmc.org,OH,CINCINNATI,True,[None],03501071438,lakeshorepharmacy94@gmail.com,No Usage,False,False
2,MD,None,Emergency Medicine,https://www.cincinnatichildrens.org/bio/b/whi...,Whitney K. Bryant,https://www.cincinnatichildrens.org/bio/b/whi...,whitney.bryant@cchmc.org,OH,CINCINNATI,True,[None],03501071438,whitney.bryant@gmail.com,Preferred E-mail,True,False
3,MD\r\n,"Medical Director, Fairfield Primary Care /Staf...",Pediatrics,https://www.cincinnatichildrens.org/bio/m/joh...,John F. Morehous,https://www.cincinnatichildrens.org/bio/m/joh...,john.morehous@cchmc.org,OH,CINCINNATI,True,"[ Fairfield Primary Care , General and Commu...",00102000285,morehous@gmail.com,No Usage,False,False
4,MD,"Attending Physician, Division of Hospital Medi...",Pediatrics,https://www.cincinnatichildrens.org/bio/a/kat...,Katherine A. S. Auger,https://www.cincinnatichildrens.org/bio/a/kat...,katherine.auger@cchmc.org,OH,CINCINNATI,True,"[ Hospital Medicine, Chair of Outcomes Pedia...",00102050711,kasauger@gmail.com,SSO Email ...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,MD\r\n,"Director, Interdisciplinary Feeding Team, FEES...",Pediatrics,https://www.cincinnatichildrens.org/bio/w/paul...,J. Paul Willging,https://www.cincinnatichildrens.org/bio/w/paul...,paul.willging@cchmc.org,OH,CINCINNATI,True,['Interdisciplinary Feeding TeamFEES Clinic an...,03841851853,paul.willging@cchmc.org,Single Sign-On,False,True
932,MD\r\n,"Director, Interdisciplinary Feeding Team, FEES...",Pediatrics,https://www.cincinnatichildrens.org/bio/w/paul...,J. Paul Willging,https://www.cincinnatichildrens.org/bio/w/paul...,paul.willging@cchmc.org,OH,CINCINNATI,True,['Interdisciplinary Feeding TeamFEES Clinic an...,03841851853,tracypierson@aol.com,Preferred E-mail,True,False
933,MD\r\n,Burnet Campus,Pediatrics,https://www.cincinnatichildrens.org/bio/w/pam-...,Pam Williams-Arya,https://www.cincinnatichildrens.org/bio/w/pam-...,pam.williams-arya@cchmc.org,OH,CINCINNATI,True,None,05501960421,pam.williams-arya@cchmc.org,No Usage,False,True
934,MD\r\n,Burnet Campus,Pediatrics,https://www.cincinnatichildrens.org/bio/w/pam-...,Pam Williams-Arya,https://www.cincinnatichildrens.org/bio/w/pam-...,pam.williams-arya@cchmc.org,OH,CINCINNATI,True,None,05501960421,pamelawa@yahoo.com,Preferred E-mail,True,False


In [101]:
with pd.ExcelWriter("../../Data/Email/HOSPITAL_EMAILS.xlsx") as writer:  
        UCLA_EMAILS.to_excel(writer, sheet_name='UCLA', index=False)
        MAYO_EMAILS.to_excel(writer, sheet_name='MAYO', index=False)
        UT_EMAILS.to_excel(writer, sheet_name='UT', index=False)